# SQL ПЛЭЙБУК: Полный справочник для собеседований и работы

**PostgreSQL** — стандарт для аналитики + явные различия с MySQL.

## СОДЕРЖАНИЕ
[1. Фундаментальные основы](#РАЗДЕЛ-1:-Фундаментальные-основы)  
[2. Выборка и фильтрация данных](#раздел-2-выборка-и-фильтрация-данных)  
[3. Агрегация и группировка](#раздел-3-агрегация-и-группировка)  
[4. Соединения таблиц (JOINs)](#раздел-4-соединения-таблиц-joins)  
[5. Подзапросы (Subqueries)](#раздел-5-подзапросы-subqueries)  
[6. CTE (Common Table Expressions)](#раздел-6-cte-common-table-expressions)  
[7. Оконные функции](#раздел-7-оконные-функции)  
[8. Оптимизация и индексы](#раздел-8-оптимизация-и-индексы)  
[9. Транзакции и управление](#раздел-9-транзакции-и-управление)  
[10. Работа с датами и строками](#раздел-10-работа-с-датами-и-строками)  
[11. Продвинутые темы](#раздел-11-продвинутые-темы)  
[12. Практические паттерны и задачи](#раздел-12-практические-паттерны-и-задачи)  
[Приложения](#приложения)

---
**Источники:** PostgreSQL docs, MySQL docs, LeetCode SQL.  
*Добавляйте свои шаблоны через PR.*

## РАЗДЕЛ 1: Фундаментальные основы

### 1.1. SQL и реляционные базы данных

**Что такое SQL?**
SQL (Structured Query Language) — декларативный язык программирования для работы с реляционными базами данных. Позволяет:
- Создавать и изменять структуру БД (DDL — Data Definition Language)
- Управлять данными (DML — Data Manipulation Language) 
- Контролировать доступ (DCL — Data Control Language)
- Управлять транзакциями (TCL — Transaction Control Language)

**Отличие SQL от MySQL?**
- **SQL** — стандартизированный язык запросов (ANSI/ISO стандарт)
- **MySQL** — конкретная СУБД (система управления базами данных), которая реализует стандарт SQL

**Реляционная база данных** — данные организованы в таблицы (отношения), связанные ключами. Основные понятия:
- Таблица (Table) — совокупность строк и столбцов
- Строка (Row/Record) — одна запись в таблице
- Столбец (Column/Field) — атрибут записи
- Первичный ключ (Primary Key) — уникальный идентификатор строки
- Внешний ключ (Foreign Key) — ссылка на первичный ключ другой таблицы

### 1.2. Типы данных

**Основные категории типов данных:**

| Категория | Примеры | Описание |
|-----------|---------|----------|
| Строковые | `CHAR`, `VARCHAR`, `TEXT` | Текстовые данные |
| Числовые | `INT`, `DECIMAL`, `FLOAT` | Числа |
| Дата/время | `DATE`, `TIME`, `TIMESTAMP` | Даты и время |
| Логические | `BOOLEAN` | Логические значения |
| Бинарные | `BLOB`, `BINARY` | Бинарные данные |
| JSON | `JSON`, `JSONB` | JSON документы |

**Разница между CHAR и VARCHAR**
```sql
-- CHAR - фиксированная длина (дополняется пробелами)
CREATE TABLE example_char (code CHAR(10));  -- 'ABC' -> 'ABC       '

-- VARCHAR - переменная длина (хранится только фактический размер)  
CREATE TABLE example_varchar (name VARCHAR(100));  -- 'Иван' -> 'Иван'

-- Рекомендации:
-- Использовать VARCHAR для текстовых полей
-- CHAR только для кодов фиксированной длины (ISO коды, аббревиатуры)
```
**Выбор типа данных:**
```sql
- INT для целых чисел (id, количество)
- DECIMAL(p,s) для денежных значений (p — точность, s — масштаб)
- VARCHAR(n) для текста, где n — максимальная длина
- DATE для дат без времени
- TIMESTAMP для даты и времени
- JSON / JSONB для неструктурированных данных (PostgreSQL: JSONB для бинарного хранения)
```

### **1.3. Ограничения (Constraints)**

#### **Что такое `PRIMARY KEY`?**
Первичный ключ — столбец (или комбинация столбцов), уникально идентифицирующий каждую строку:
* Гарантирует **уникальность** значений
* Не допускает **NULL** значений
* Автоматически создает индекс (обычно кластеризованный)
* В таблице может быть только **один** `PRIMARY KEY`

```sql
-- Создание первичного ключа
CREATE TABLE users (
    id INT PRIMARY KEY,
    email VARCHAR(100) NOT NULL UNIQUE
);
CREATE TABLE orders (
    order_id INT,
    user_id INT,
    PRIMARY KEY (order_id, user_id)
);
```

#### **Что такое `FOREIGN KEY`?**
Внешний ключ — ограничение, обеспечивающее ссылочную целостность между таблицами:
* Связывает поле в дочерней таблице с `PRIMARY KEY` в родительской
* Предотвращает удаление связанных данных (`CASCADE`, `SET NULL`, `RESTRICT`)
* Обеспечивает целостность отношений

```sql
-- Создание внешнего ключа
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    user_id INT,
    FOREIGN KEY (user_id) REFERENCES users(id)
        ON DELETE CASCADE
        ON UPDATE CASCADE
);
-- Проверка внешних ключей (MySQL)
SHOW CREATE TABLE orders;
```

#### **Типы ограничений:**
| Ограничение | Описание | Синтаксис |
| :--- | :--- | :--- |
| `NOT NULL` | Запрещает NULL | `name VARCHAR(50) NOT NULL` |
| `UNIQUE` | Гарантирует уникальность | `email VARCHAR(100) UNIQUE` |
| `PRIMARY KEY` | Первичный ключ | `id INT PRIMARY KEY` |
| `FOREIGN KEY` | Внешний ключ | `FOREIGN KEY (col) REFERENCES table(col)` |
| `CHECK` | Проверка условия | `age INT CHECK (age >= 18)` |
| `DEFAULT` | Значение по умолчанию | `created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP` |

#### **Практический пример:**
```sql
CREATE TABLE employees (
    id INT PRIMARY KEY AUTO_INCREMENT,
    employee_code CHAR(10) UNIQUE NOT NULL,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    age INT CHECK (age >= 18 AND age <= 70),
    department_id INT,
    salary DECIMAL(10,2) DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (department_id) REFERENCES departments(id)
        ON DELETE SET NULL
);
```

### **1.4. CRUD операции**

#### **`CREATE` — создание таблиц**

```sql
-- Базовая структура
CREATE TABLE table_name (
    column1 datatype constraints,
    column2 datatype constraints,
    ...
);

-- Пример
CREATE TABLE products (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    price DECIMAL(10,2) CHECK (price >= 0),
    category VARCHAR(50),
    in_stock BOOLEAN DEFAULT TRUE,
    created_at DATETIME DEFAULT NOW()
);

-- Копирование структуры
CREATE TABLE products_backup LIKE products;  -- MySQL
CREATE TABLE products_backup AS SELECT * FROM products WHERE 1=0;  -- кросс-платформенный
```

#### **`ALTER` — изменение таблиц**

```sql
-- Добавить столбец
ALTER TABLE employees ADD COLUMN birth_date DATE;

-- Изменить тип столбца
ALTER TABLE employees MODIFY COLUMN name VARCHAR(150);  -- MySQL
ALTER TABLE employees ALTER COLUMN name TYPE VARCHAR(150);  -- PostgreSQL

-- Удалить столбец  
ALTER TABLE employees DROP COLUMN middle_name;

-- Добавить ограничение
ALTER TABLE employees ADD CONSTRAINT chk_age CHECK (age >= 18);

-- Переименовать таблицу
ALTER TABLE old_name RENAME TO new_name;
```

#### **`INSERT` — добавление данных**

```sql
-- Вставка одной строки
INSERT INTO users (name, email, age) 
VALUES ('Иван Иванов', 'ivan@example.com', 30);

-- Вставка нескольких строк
INSERT INTO products (name, price, category) VALUES
    ('Ноутбук', 1500.00, 'Электроника'),
    ('Стул', 200.00, 'Мебель'),
    ('Книга', 30.00, 'Книги');

-- Вставка из другой таблицы
INSERT INTO users_archive (id, name, email)
SELECT id, name, email 
FROM users 
WHERE created_at < '2023-01-01';
```

#### **`SELECT` — выборка данных**

```sql
-- Получить все данные
SELECT * FROM employees;

-- Выбрать конкретные столбцы
SELECT name, email, salary FROM employees;

-- Выбрать уникальные значения
SELECT DISTINCT department FROM employees;
```

#### **`UPDATE` — обновление данных**

```sql
-- Обновить все строки
UPDATE products SET price = price * 1.1;  -- увеличить все цены на 10%

-- Обновить с условием
UPDATE employees 
SET salary = salary * 1.15, 
    updated_at = NOW()
WHERE department = 'IT' 
  AND performance_rating > 4.0;

-- Обновить из другой таблицы (MySQL)
UPDATE employees e
JOIN salary_changes s ON e.id = s.employee_id
SET e.salary = s.new_salary
WHERE s.effective_date = '2025-01-01';
```

#### **`DELETE` — удаление данных**

```sql
-- Удалить с условием
DELETE FROM orders 
WHERE status = 'cancelled' 
  AND created_at < '2024-01-01';

-- Удалить все данные (осторожно!)
DELETE FROM temp_logs;  -- можно откатить, медленно
```

### **1.5. Разница между DELETE, TRUNCATE и DROP**

#### **Сравнение операций:**

| Операция | Скорость | Возврат (ROLLBACK) | Автоинкремент | Где использовать |
| :--- | :--- | :--- | :--- | :--- |
| `DELETE` | Медленно | **Да** | Сохраняется | Удаление строк с условием `WHERE` |
| `TRUNCATE` | Быстро | **Нет*** | Сбрасывается | Удаление **ВСЕХ** строк, сброс таблицы |
| `DROP` | Мгновенно | Нет | Удаляется | Удаление всей таблицы **со структурой** |

\* В PostgreSQL `TRUNCATE` можно откатить, если он выполнен внутри транзакции.

#### **Примеры команд:**

```sql
-- DELETE: удаление с условием, можно откатить
BEGIN TRANSACTION;
DELETE FROM users WHERE age < 18;
-- ROLLBACK; -- откатить удаление
COMMIT;

-- TRUNCATE: удаление всех данных, быстро (нельзя использовать WHERE)
TRUNCATE TABLE session_logs;

-- DROP: удаление таблицы полностью (удаляет и данные, и структуру)
DROP TABLE temporary_data;

-- Каскадное удаление
DROP TABLE users CASCADE;  -- удаляет таблицу 'users' и все зависимые от неё объекты
```

#### **Важные нюансы:**

*   **`TRUNCATE`** — это операция **DDL** (Data Definition Language), а не DML (как `DELETE`).
*   **`DELETE`** активирует триггеры удаления, **`TRUNCATE`** — **нет**.
*   **`TRUNCATE`** обычно требует больших привилегий (часто доступно только у администратора БД - DBA).
*   В **PostgreSQL** `TRUNCATE` можно откатить (`ROLLBACK`), если он выполнен внутри транзакции.

### **Антипаттерны раздела 1**

#### ❌ **Не использовать первичные ключи**

```sql
-- ПЛОХО: таблица без первичного ключа
CREATE TABLE logs (
    message TEXT,
    created_at TIMESTAMP
);
-- Проблемы: дубликаты, сложность обновления, медленные JOIN

-- ✅ ХОРОШО: всегда добавлять первичный ключ
CREATE TABLE logs (
    id INT PRIMARY KEY AUTO_INCREMENT,
    message TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

#### ❌ **Использовать VARCHAR без указания длины**

```sql
-- ПЛОХО: неограниченная длина (MySQL создаст TEXT)
CREATE TABLE users (
    name VARCHAR,  -- ошибка в MySQL, требуется указать длину
    email VARCHAR
);

-- ✅ ХОРОШО: указывать разумную длину
CREATE TABLE users (
    name VARCHAR(100),
    email VARCHAR(150)
);
```

#### ❌ **Хранить несколько значений в одном поле**

```sql
-- ПЛОХО: нарушение первой нормальной формы
CREATE TABLE orders (
    id INT PRIMARY KEY,
    product_ids VARCHAR(255)  -- '1,5,23,42'
);

-- ✅ ХОРОШО: использовать связующую таблицу
CREATE TABLE orders (
    id INT PRIMARY KEY
);

CREATE TABLE order_items (
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);
```

#### ❌ **Использовать SELECT * в продакшене**

```sql
-- ПЛОХО: выбирать все столбцы
SELECT * FROM users WHERE id = 1;

-- ✅ ХОРОШО: выбирать только нужные столбцы
SELECT id, name, email FROM users WHERE id = 1;
-- Преимущества: меньше данных по сети, лучше кэширование, индексы работают эффективнее
```

### **Лучшие практики раздела 1**

#### ✅ **Всегда использовать первичные ключи**
*   Даже для логов и временных данных
*   Автоинкремент для таблиц с частыми вставками
*   UUID для распределенных систем

#### ✅ **Выбирать подходящие типы данных**
*   `INT` для счетчиков и идентификаторов
*   `DECIMAL` для денежных значений (не `FLOAT`)
*   `VARCHAR` с разумной длиной для текста
*   `DATETIME`/`TIMESTAMP` для меток времени

#### ✅ **Использовать ограничения на уровне БД**
*   `NOT NULL` для обязательных полей
*   `UNIQUE` для email, телефонных номеров
*   `FOREIGN KEY` для поддержания целостности
*   `CHECK` для валидации бизнес-правил

#### ✅ **Следовать соглашениям об именовании**

```sql
-- Пример хороших имен:
users                -- таблицы во множественном числе
id                   -- первичный ключ
user_id              -- внешний ключ
created_at, updated_at  -- временные метки
is_active, has_access  -- булевы флаги
```

### **Различия MySQL и PostgreSQL**

| Концепция | MySQL | PostgreSQL |
| :--- | :--- | :--- |
| **Тип строки по умолчанию** | `VARCHAR(255)` | Без ограничения длины |
| **Логический тип** | `BOOLEAN` (синоним `TINYINT(1)`) | Нативный `BOOLEAN` |
| **Автоинкремент** | `AUTO_INCREMENT` | `SERIAL` или `GENERATED AS IDENTITY` |
| **Каскадное удаление** | `ON DELETE CASCADE` | `ON DELETE CASCADE` |
| **Проверка длины строки** | `CHECK(LENGTH(name) > 0)` | То же самое |

### **Шпаргалка раздела 1**

#### **Создание таблицы:**

```sql
CREATE TABLE table_name (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(150) UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

#### **Основные типы данных:**
*   **`INT`, `BIGINT`** — целые числа
*   **`DECIMAL(10,2)`** — числа с фиксированной точностью
*   **`VARCHAR(n)`** — текст переменной длины
*   **`TEXT`** — длинный текст
*   **`DATE`, `DATETIME`, `TIMESTAMP`** — дата и время
*   **`BOOLEAN`** — истина/ложь

#### **Ограничения:**
*   **`PRIMARY KEY`** — уникальный идентификатор
*   **`FOREIGN KEY`** — связь между таблицами
*   **`NOT NULL`** — обязательное поле
*   **`UNIQUE`** — уникальные значения
*   **`CHECK`** — проверка условия
*   **`DEFAULT`** — значение по умолчанию

#### **Операции с данными:**

```sql
INSERT INTO table (col1, col2) VALUES (val1, val2);
SELECT * FROM table WHERE condition;
UPDATE table SET col = value WHERE condition;
DELETE FROM table WHERE condition;
```

## РАЗДЕЛ 2: Выборка и фильтрация данных

### **2.1. WHERE, ORDER BY, LIMIT/OFFSET**

#### **Базовая структура запроса SELECT**

```sql
SELECT column1, column2, ...        -- Что выбрать
FROM table_name                     -- Откуда выбрать
WHERE condition                     -- Условие фильтрации строк
ORDER BY column1 [ASC|DESC]         -- Сортировка результатов
LIMIT number_of_rows;               -- Ограничение количества строк
```

#### **Ключевое слово WHERE**
Фильтрует строки до их обработки и агрегации.

```sql
-- Простые условия
SELECT * FROM employees WHERE salary > 50000;
SELECT * FROM products WHERE price BETWEEN 20 AND 100;
SELECT * FROM users WHERE country IN ('Россия', 'Беларусь', 'Казахстан');

-- Комбинирование условий
SELECT * FROM orders 
WHERE status = 'completed' 
  AND total_amount > 1000 
  AND order_date >= '2024-01-01';
```

#### **Сортировка с ORDER BY**

```sql
-- Сортировка по одному столбцу (по возрастанию ASC по умолчанию)
SELECT name, salary FROM employees ORDER BY salary;

-- Сортировка по убыванию
SELECT name, salary FROM employees ORDER BY salary DESC;

-- Сортировка по нескольким столбцам
SELECT first_name, last_name, department, salary 
FROM employees 
ORDER BY department ASC, salary DESC;

-- Сортировка по выражению или номеру столбца (не рекомендуется для продакшена)
SELECT name, salary, salary * 0.87 AS salary_after_tax
FROM employees 
ORDER BY 3 DESC;  -- Сортировка по 3-му столбцу (salary_after_tax)
```

#### **Ограничение результатов LIMIT и OFFSET**

```sql
-- Получить первые 10 записей
SELECT * FROM products ORDER BY price DESC LIMIT 10;

-- Пагинация: пропустить 20 записей, взять следующие 10
SELECT * FROM products 
ORDER BY created_at DESC 
LIMIT 10 OFFSET 20;  -- Страница 3 при размере страницы 10

-- Альтернативный синтаксис (MySQL)
SELECT * FROM products LIMIT 20, 10;  -- OFFSET 20, LIMIT 10
```

#### **Порядок выполнения запроса для понимания:**
1.  **`FROM`** — определение таблицы
2.  **`WHERE`** — фильтрация строк
3.  **`SELECT`** — выбор столбцов
4.  **`ORDER BY`** — сортировка
5.  **`LIMIT`/`OFFSET`** — ограничение вывода

### **2.2. Операторы сравнения и логические операторы**

#### **Операторы сравнения:**

| Оператор | Описание | Пример |
| :--- | :--- | :--- |
| `=` | Равно | `WHERE status = 'active'` |
| `<>` или `!=` | Не равно | `WHERE status <> 'deleted'` |
| `<` | Меньше | `WHERE age < 18` |
| `>` | Больше | `WHERE salary > 50000` |
| `<=` | Меньше или равно | `WHERE quantity <= 10` |
| `>=` | Больше или равно | `WHERE score >= 60` |
| `BETWEEN` | В диапазоне (включительно) | `WHERE price BETWEEN 10 AND 100` |
| `IN` | В списке значений | `WHERE country IN ('RU', 'BY', 'KZ')` |
| `LIKE` | Соответствие шаблону | `WHERE name LIKE 'Ив%'` |
| `IS NULL` | Проверка на NULL | `WHERE email IS NULL` |

#### **Логические операторы:**

```sql
-- AND: оба условия истинны
SELECT * FROM users WHERE age >= 18 AND age <= 65;

-- OR: хотя бы одно условие истинно
SELECT * FROM products WHERE category = 'Электроника' OR price > 1000;

-- NOT: отрицание условия
SELECT * FROM orders WHERE NOT status = 'cancelled';

-- Комбинации с использованием скобок для контроля приоритета
SELECT * FROM employees 
WHERE (department = 'IT' AND salary > 80000)
   OR (department = 'Sales' AND commission > 0.1);
```

#### **Приоритет операторов (от высшего к низшему):**
1.  **Скобки `()`**
2.  **Операторы сравнения** (`=`, `<>`, `<`, `>`, и т.д.)
3.  **`NOT`**
4.  **`AND`**
5.  **`OR`**

### **2.3. LIKE и регулярные выражения**

#### **Оператор LIKE для поиска по шаблону:**

```sql
-- %: любое количество любых символов (0 или более)
SELECT * FROM users WHERE name LIKE 'Ив%';     -- Начинается на "Ив" (Иван, Игорь)
SELECT * FROM users WHERE email LIKE '%gmail.com';  -- Заканчивается на "gmail.com"
SELECT * FROM products WHERE name LIKE '%ноутбук%'; -- Содержит "ноутбук"

-- _: один любой символ
SELECT * FROM users WHERE phone LIKE '+7___%';  -- Российские номера (после +7 три любые цифры)
SELECT * FROM products WHERE sku LIKE 'ABC_2024'; -- ABC12024, ABC22024, и т.д.

-- Экранирование специальных символов
SELECT * FROM logs WHERE message LIKE '100\% complete';  -- Поиск "100% complete"
```

#### **Регулярные выражения (REGEXP):**

```sql
-- MySQL: REGEXP
SELECT * FROM users WHERE email REGEXP '^[a-z]+@gmail\.com$';  -- только gmail
SELECT * FROM products WHERE name REGEXP '^(Pro|Premium)';  -- начинается с Pro или Premium

-- PostgreSQL: ~ (регистрозависимый) и ~* (регистронезависимый)
SELECT * FROM users WHERE email ~ '^[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}$';
SELECT * FROM products WHERE name ~* 'iphone.*(14|15)';  -- iPhone 14 или 15, регистронезависимо

-- Проверка на соответствие шаблону
SELECT 'hello' REGEXP '^h.*o$';  -- MySQL: возвращает 1 (true)
SELECT 'hello' ~ '^h.*o$';       -- PostgreSQL: возвращает true
```

### **2.4. Работа с NULL значениями**

#### **Что такое NULL?**
*   **NULL** означает "отсутствие значения" или "неизвестное значение".
*   **NULL ≠ 0**, NULL ≠ пустая строка `''`, NULL ≠ false.
*   Любая операция с NULL возвращает NULL (`NULL + 5 = NULL`, `NULL = NULL` → `NULL`).

#### **Проверка на NULL:**

```sql
-- ❌ НЕПРАВИЛЬНО (всегда возвращает NULL, что интерпретируется как false)
SELECT * FROM employees WHERE phone = NULL;  -- Не найдет ни одной строки
SELECT * FROM employees WHERE phone != NULL; -- Не найдет ни одной строки

-- ✅ ПРАВИЛЬНО: использовать IS NULL или IS NOT NULL
SELECT * FROM employees WHERE phone IS NULL;
SELECT * FROM employees WHERE phone IS NOT NULL;
```

#### **Функции для работы с NULL:**

```sql
-- COALESCE: возвращает первое не-NULL значение
SELECT 
    name, 
    COALESCE(phone, 'Не указан') AS phone_display,
    COALESCE(email, phone, 'Нет контактов') AS contact
FROM users;

-- IFNULL (MySQL) / ISNULL (SQL Server): аналог COALESCE для двух аргументов
SELECT name, IFNULL(phone, 'N/A') FROM users;  -- MySQL

-- NULLIF: возвращает NULL если значения равны, иначе первое значение
SELECT NULLIF(sale_price, 0) AS effective_price FROM products;  -- Заменяет 0 на NULL

-- Пример практического использования
SELECT 
    COUNT(*) AS total_users,
    COUNT(phone) AS users_with_phone,  -- COUNT игнорирует NULL
    COUNT(DISTINCT email) AS unique_emails
FROM users;
```

#### **NULL в агрегатных функциях:**

```sql
-- Агрегатные функции игнорируют NULL (кроме COUNT(*))
SELECT 
    AVG(commission) AS avg_commission,           -- NULL не учитываются
    SUM(bonus) AS total_bonus,                   -- NULL игнорируются
    MIN(salary) AS min_salary,                   -- NULL игнорируются
    MAX(salary) AS max_salary,                   -- NULL игнорируются
    COUNT(commission) AS employees_with_commission,  -- Только не-NULL
    COUNT(*) AS total_employees                  -- Все строки, включая NULL
FROM employees;
```

#### **Сортировка NULL значений:**

```sql
-- По умолчанию NULL считается наименьшим значением (идут первыми при ASC)
SELECT * FROM products ORDER BY price ASC;  -- NULL будут первыми

-- Явное указание расположения NULL
SELECT * FROM employees 
ORDER BY commission DESC NULLS LAST;  -- NULL в конце

SELECT * FROM employees 
ORDER BY commission ASC NULLS FIRST;  -- NULL в начале (по умолчанию)
```

### **Антипаттерны раздела 2**

#### ❌ **Использование функций в WHERE на индексированных полях**

```sql
-- ПЛОХО: индекс не используется
SELECT * FROM users WHERE YEAR(created_at) = 2024;
SELECT * FROM users WHERE UPPER(name) = 'ИВАН';

-- ✅ ХОРОШО: переписать без функций
SELECT * FROM users WHERE created_at >= '2024-01-01' AND created_at < '2025-01-01';
SELECT * FROM users WHERE name = 'Иван';  -- Если нужен регистронезависимый поиск, используйте COLLATE
```

#### ❌ **Неправильная проверка диапазонов дат**

```sql
-- ПЛОХО: преобразование типов, индекс не используется
SELECT * FROM orders WHERE DATE(order_datetime) = '2024-01-15';

-- ✅ ХОРОШО: использовать диапазон
SELECT * FROM orders 
WHERE order_datetime >= '2024-01-15 00:00:00' 
  AND order_datetime < '2024-01-16 00:00:00';
```

#### ❌ **Использование OR вместо IN для многих значений**

```sql
-- ПЛОХО: нечитаемо, может быть медленнее
SELECT * FROM products 
WHERE category = 'Электроника' 
   OR category = 'Бытовая техника'
   OR category = 'Мебель'
   OR category = 'Книги';

-- ✅ ХОРОШО: использовать IN
SELECT * FROM products 
WHERE category IN ('Электроника', 'Бытовая техника', 'Мебель', 'Книги');

-- Еще лучше: использовать временную таблицу или таблицу-справочник
```

#### ❌ **SELECT * в больших таблицах**

```sql
-- ПЛОХО: выборка всех столбцов, включая тяжелые (TEXT, BLOB)
SELECT * FROM products WHERE category = 'Электроника';

-- ✅ ХОРОШО: выбирать только необходимые столбцы
SELECT id, name, price, category 
FROM products 
WHERE category = 'Электроника';
```

### **Лучшие практики раздела 2**

#### ✅ **Всегда использовать ORDER BY с LIMIT**

```sql
-- Без ORDER BY результат непредсказуем
SELECT * FROM products LIMIT 10;  -- ❌ Какие 10? Случайные!

-- С ORDER BY результат детерминирован
SELECT * FROM products ORDER BY created_at DESC LIMIT 10;  -- ✅ Последние 10
```

#### ✅ **Правильное экранирование пользовательского ввода**

```sql
-- Защита от SQL-инъекций
-- ❌ ОПАСНО (SQL-инъекция возможна):
"SELECT * FROM users WHERE name = '" + user_input + "'"

-- ✅ БЕЗОПАСНО: использовать параметризованные запросы
-- В приложении:
cursor.execute("SELECT * FROM users WHERE name = %s", (user_input,))

-- Или использовать ORM, который экранирует ввод автоматически
```

#### ✅ **Использование EXPLAIN для анализа запросов**

```sql
-- Перед оптимизацией всегда смотрите план выполнения
EXPLAIN SELECT * FROM users WHERE email LIKE '%@gmail.com';

-- В MySQL можно добавить FORMAT=JSON для детальной информации
EXPLAIN FORMAT=JSON SELECT * FROM large_table WHERE date_column > '2024-01-01';
```

#### ✅ **Создание индексов для часто фильтруемых полей**

```sql
-- Для ускорения WHERE и ORDER BY
CREATE INDEX idx_users_email ON users(email);
CREATE INDEX idx_orders_date_status ON orders(order_date, status);
CREATE INDEX idx_products_category_price ON products(category, price DESC);

-- Для ускорения поиска по префиксу
CREATE INDEX idx_users_name_prefix ON users(name(10));  -- Первые 10 символов (MySQL)
```

### **Различия MySQL и PostgreSQL**

| Операция | MySQL | PostgreSQL |
| :--- | :--- | :--- |
| **Конкатенация строк** | `CONCAT('A', 'B')` или `'A' ‖ 'B'` с `SET sql_mode='PIPES_AS_CONCAT'` | `'A' ‖ 'B'` или `CONCAT('A', 'B')` |
| **Регулярные выражения** | `REGEXP`, `RLIKE` | `~`, `~*`, `!~`, `!~*` |
| **LIMIT/OFFSET** | `LIMIT 10 OFFSET 20` или `LIMIT 20, 10` | Только `LIMIT 10 OFFSET 20` |`
| **Проверка на NULL в WHERE** | `WHERE column IS NULL` | `WHERE column IS NULL` |
| **Сортировка NULL** | `ORDER BY column ASC` (NULL первые) | `ORDER BY column ASC NULLS FIRST\|LAST` |
| **LIKE с escape** | `LIKE '100\%' ESCAPE '\'` | `LIKE '100\%' ESCAPE '\'` |
| **ILIKE (регистронезависимый)** | Не поддерживается, использовать `LIKE` или `REGEXP` | Поддерживается `ILIKE` |

### **Шпаргалка раздела 2**

#### **Базовый запрос с фильтрацией:**

```sql
SELECT column1, column2
FROM table
WHERE condition
ORDER BY column1 DESC
LIMIT 10 OFFSET 20;
```

#### **Основные операторы WHERE:**
*   **`=`, `<>`, `<`, `>`, `<=`, `>=`** — сравнение
*   **`BETWEEN a AND b`** — диапазон (включительно)
*   **`IN (val1, val2, ...)`** — значение в списке
*   **`LIKE 'pattern%'`** — поиск по шаблону (`%` любое количество, `_` один символ)
*   **`IS NULL`, `IS NOT NULL`** — проверка на NULL

#### **Логические операторы:**
*   **`AND`** — оба условия истинны
*   **`OR`** — хотя бы одно условие истинно
*   **`NOT`** — отрицание условия
*   **Приоритет:** `()` > `NOT` > `AND` > `OR`

#### **Функции работы с NULL:**
*   **`COALESCE(val1, val2, ...)`** — первое не-NULL значение
*   **`IFNULL(val, default)`** (MySQL) — заменяет NULL на default
*   **`NULLIF(val1, val2)`** — возвращает NULL если значения равны

#### **Важные замечания:**
*   Всегда используйте `ORDER BY` с `LIMIT`
*   Избегайте функций в `WHERE` на индексированных полях
*   Для проверки на NULL используйте `IS NULL`, а не `= NULL`
*   Выбирайте только нужные столбцы, а не `SELECT *`
*   Используйте `EXPLAIN` для анализа медленных запросов

## РАЗДЕЛ 3: Агрегация и группировка

### 3.1. Агрегатные функции (COUNT, SUM, AVG, MIN, MAX)

**Что такое агрегатные функции?**
Функции, которые выполняют вычисление на наборе строк и возвращают **одно** значение. Они "сворачивают" множество строк в одну.

**Основные агрегатные функции:**

| Функция | Описание | Особенности работы с NULL |
|---------|----------|---------------------------|
| `COUNT()` | Подсчет количества строк | `COUNT(*)` считает все строки, `COUNT(column)` игнорирует NULL |
| `SUM()` | Сумма значений | Игнорирует NULL значения |
| `AVG()` | Среднее значение | Игнорирует NULL, для пустого набора возвращает NULL |
| `MIN()` | Минимальное значение | Игнорирует NULL |
| `MAX()` | Максимальное значение | Игнорирует NULL |
| `GROUP_CONCAT()` (MySQL) / `STRING_AGG()` (PG) | Объединение строк | Конкатенирует значения в строку |

**Примеры использования:**
```sql
-- Базовая статистика по таблице
SELECT 
    COUNT(*) AS total_orders,
    SUM(amount) AS total_revenue,
    AVG(amount) AS avg_order_value,
    MIN(amount) AS min_order,
    MAX(amount) AS max_order
FROM orders;

-- COUNT с DISTINCT (уникальные значения)
SELECT 
    COUNT(DISTINCT customer_id) AS unique_customers,
    COUNT(DISTINCT product_id) AS unique_products
FROM orders;

-- Объединение значений в строку (MySQL)
SELECT 
    department,
    GROUP_CONCAT(DISTINCT name ORDER BY name SEPARATOR ', ') AS employees
FROM employees
GROUP BY department;

-- Объединение значений (PostgreSQL)
SELECT 
    department,
    STRING_AGG(DISTINCT name, ', ' ORDER BY name) AS employees
FROM employees
GROUP BY department;
```
#### **COUNT(*) vs COUNT(column)**

```sql
-- Таблица с данными:
-- id | name | phone
-- 1  | Иван | +7...
-- 2  | Анна | NULL
-- 3  | NULL | +7...

SELECT 
    COUNT(*) AS total_rows,           -- 3 (все строки)
    COUNT(name) AS names_count,       -- 2 (только не-NULL в name)
    COUNT(phone) AS phones_count,     -- 2 (только не-NULL в phone)
    COUNT(DISTINCT name) AS unique_names  -- 2 (Иван, Анна)
FROM users;
```

### **3.2. GROUP BY и HAVING**

#### **GROUP BY — группировка строк**
Группирует строки с одинаковыми значениями в указанных столбцах для применения агрегатных функций к каждой группе.

```sql
-- Группировка по одному столбцу
SELECT 
    department,
    COUNT(*) AS employee_count,
    AVG(salary) AS avg_salary
FROM employees
GROUP BY department;

-- Группировка по нескольким столбцам
SELECT 
    department,
    job_title,
    COUNT(*) AS count,
    SUM(salary) AS total_salary
FROM employees
GROUP BY department, job_title;

-- Группировка с выражениями
SELECT 
    EXTRACT(YEAR FROM hire_date) AS hire_year,
    COUNT(*) AS hires_count
FROM employees
GROUP BY EXTRACT(YEAR FROM hire_date)
ORDER BY hire_year;
```

#### **HAVING — фильтрация после группировки**
Фильтрует результаты агрегации (группы), тогда как WHERE фильтрует строки до группировки.

```sql
-- Найти отделы с более чем 10 сотрудниками
SELECT 
    department,
    COUNT(*) AS employee_count
FROM employees
GROUP BY department
HAVING COUNT(*) > 10;

-- Отделы со средней зарплатой выше 70000
SELECT 
    department,
    AVG(salary) AS avg_salary
FROM employees
GROUP BY department
HAVING AVG(salary) > 70000
ORDER BY avg_salary DESC;

-- Комбинирование WHERE и HAVING
SELECT 
    department,
    AVG(salary) AS avg_salary,
    COUNT(*) AS emp_count
FROM employees
WHERE hire_date >= '2020-01-01'   -- Фильтр строк ДО группировки
GROUP BY department
HAVING AVG(salary) > 50000        -- Фильтр групп ПОСЛЕ группировки
   AND COUNT(*) >= 5;             -- Можно использовать несколько условий
```

### **3.3. Фильтрация до и после агрегации**

#### **Порядок выполнения запроса с GROUP BY:**
1.  **`FROM`** — получение данных из таблицы
2.  **`WHERE`** — фильтрация строк (до группировки)
3.  **`GROUP BY`** — группировка строк
4.  **`HAVING`** — фильтрация групп (после группировки)
5.  **`SELECT`** — выбор столбцов и вычисление агрегатов
6.  **`ORDER BY`** — сортировка
7.  **`LIMIT`** — ограничение

```sql
-- Пример, показывающий разницу WHERE и HAVING
SELECT 
    department,
    COUNT(*) AS total,
    AVG(salary) AS avg_salary
FROM employees
WHERE salary > 30000           -- Исключаем низкооплачиваемых ДО подсчета среднего
GROUP BY department
HAVING AVG(salary) > 50000    -- Исключаем отделы с низкой средней ПОСЛЕ подсчета
ORDER BY avg_salary DESC;

-- Практический пример: анализ продаж
SELECT 
    p.category,
    COUNT(DISTINCT o.customer_id) AS unique_customers,
    SUM(o.quantity) AS total_quantity,
    SUM(o.quantity * p.price) AS total_revenue,
    AVG(o.quantity * p.price) AS avg_order_value
FROM orders o
JOIN products p ON o.product_id = p.id
WHERE o.order_date >= '2024-01-01'   -- Только заказы 2024 года
  AND o.status = 'completed'         -- Только завершенные
GROUP BY p.category
HAVING SUM(o.quantity * p.price) > 10000  -- Категории с выручкой > 10000
ORDER BY total_revenue DESC;
```

### **3.4. DISTINCT и GROUP BY для уникальных значений**

#### **DISTINCT vs GROUP BY:**

```sql
-- DISTINCT: только уникальные значения (без агрегации)
SELECT DISTINCT department FROM employees;

-- GROUP BY: уникальные значения + возможность агрегации
SELECT department FROM employees GROUP BY department;

-- Оба дают одинаковый результат для уникальности,
-- но GROUP BY более гибкий (можно добавить агрегатные функции)

-- Пример разницы:
-- DISTINCT с агрегацией (считает по всей таблице)
SELECT COUNT(DISTINCT department) AS dept_count FROM employees;

-- GROUP BY с агрегацией (считает по группам)
SELECT department, COUNT(*) FROM employees GROUP BY department;
```

#### **Удаление дубликатов:**

```sql
-- Найти дубликаты по email
SELECT email, COUNT(*) AS duplicate_count
FROM users
GROUP BY email
HAVING COUNT(*) > 1;

-- Удалить дубликаты, оставив самую новую запись
DELETE FROM users
WHERE id NOT IN (
    SELECT MIN(id)
    FROM users
    GROUP BY email
);

-- Альтернативный способ с временной таблицей
CREATE TABLE users_dedup AS
SELECT DISTINCT * FROM users;
```

### **Антипаттерны раздела 3**

#### ❌ **Смешивание агрегированных и неагрегированных столбцов без GROUP BY**

```sql
-- ❌ ПЛОХО: Ошибка в большинстве СУБД (или недетерминированный результат)
SELECT name, AVG(salary) FROM employees;

-- ✅ ХОРОШО: Добавить все неагрегированные столбцы в GROUP BY
SELECT department, AVG(salary) FROM employees GROUP BY department;

-- ИЛИ использовать оконные функции
SELECT name, salary, AVG(salary) OVER() FROM employees;
```

#### ❌ **Использование HAVING без GROUP BY для фильтрации агрегатов**

```sql
-- ❌ ПЛОХО: HAVING без GROUP BY работает, но это не интуитивно
SELECT AVG(salary) FROM employees HAVING AVG(salary) > 50000;

-- ✅ ХОРОШО: Использовать вложенный запрос
SELECT * FROM (
    SELECT AVG(salary) AS avg_salary FROM employees
) t WHERE avg_salary > 50000;
```

#### ❌ **Группировка по слишком большому числу столбцов**

```sql
-- ❌ ПЛОХО: Слишком детальная группировка, почти каждая строка — отдельная группа
SELECT first_name, last_name, email, phone, address, department, 
       COUNT(*) -- Всегда 1
FROM employees
GROUP BY first_name, last_name, email, phone, address, department;

-- ✅ ХОРОШО: Группировать только по значимым для анализа признакам
SELECT department, job_level, COUNT(*) AS employee_count
FROM employees
GROUP BY department, job_level;
```

#### ❌ **Неоптимальное использование DISTINCT**

```sql
-- ❌ ПЛОХО: DISTINCT на большом наборе данных
SELECT DISTINCT * FROM huge_log_table;

-- ✅ ХОРОШО: Сначала отфильтровать, затем применять DISTINCT
SELECT DISTINCT user_id, action 
FROM huge_log_table 
WHERE date >= '2024-01-01';

-- ИЛИ использовать GROUP BY если нужны агрегаты
SELECT user_id, action, COUNT(*) 
FROM huge_log_table 
GROUP BY user_id, action;
```

### **Лучшие практики раздела 3**

#### ✅ **Всегда включать неагрегированные столбцы в GROUP BY**

```sql
-- Правило: все столбцы в SELECT, не находящиеся внутри агрегатной функции,
-- должны быть в GROUP BY
SELECT 
    department,        -- в GROUP BY
    job_title,         -- в GROUP BY  
    COUNT(*) AS count,
    AVG(salary) AS avg_salary
FROM employees
GROUP BY department, job_title;  -- Все неагрегированные столбцы
```

#### ✅ **Фильтровать на ранних этапах с помощью WHERE**

```sql
-- Оптимизация: фильтровать до группировки, чтобы уменьшить объем данных
-- ❌ ПЛОХО: Сначала все сгруппировать, потом отфильтровать
SELECT category, AVG(price)
FROM products
GROUP BY category
HAVING AVG(price) > 100 AND category LIKE 'E%';

-- ✅ ХОРОШО: Отфильтровать строки до группировки
SELECT category, AVG(price)
FROM products
WHERE category LIKE 'E%'  -- Уменьшаем данные для группировки
GROUP BY category
HAVING AVG(price) > 100;
```

#### ✅ **Использовать составные агрегаты для комплексного анализа**

```sql
-- Анализ распределения в одном запросе
SELECT 
    department,
    COUNT(*) AS total,
    SUM(CASE WHEN salary > 70000 THEN 1 ELSE 0 END) AS high_earners,
    SUM(CASE WHEN salary < 40000 THEN 1 ELSE 0 END) AS low_earners,
    ROUND(AVG(salary), 2) AS avg_salary,
    MIN(salary) AS min_salary,
    MAX(salary) AS max_salary,
    ROUND(100.0 * SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) / COUNT(*), 2) AS female_percentage
FROM employees
GROUP BY department;
```

#### ✅ **Создавать индексы для полей в GROUP BY и WHERE**

```sql
-- Индексы для ускорения группировки
CREATE INDEX idx_employees_dept ON employees(department);
CREATE INDEX idx_orders_date_status ON orders(order_date, status);
CREATE INDEX idx_products_category_price ON products(category, price);

-- Составные индексы для часто используемых комбинаций GROUP BY
CREATE INDEX idx_sales_analysis ON sales(region, product_category, sale_date);
```

### **Различия MySQL и PostgreSQL**

| Функциональность | MySQL | PostgreSQL |
| :--- | :--- | :--- |
| **Конкатенация строк в группе** | `GROUP_CONCAT(col SEPARATOR ', ')` | `STRING_AGG(col, ', ')` |
| **Фильтр в агрегатной функции** | Не поддерживается `FILTER` | `SUM(col) FILTER (WHERE condition)` |
| **Статистические агрегаты** | `STDDEV_POP()`, `VAR_POP()` | `STDDEV()`, `VARIANCE()` |
| **Перцентили** | `PERCENTILE_CONT()` не поддерживается | `PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY col)` |
| **Режим ONLY_FULL_GROUP_BY** | По умолчанию строгий с версии 5.7 | Всегда строгий |
| **Группировка по номеру столбца** | `GROUP BY 1, 2` | Не рекомендуется, лучше по имени |

#### **Пример FILTER в PostgreSQL:**

```sql
-- Агрегация с условием для каждой функции
SELECT 
    department,
    COUNT(*) AS total,
    COUNT(*) FILTER (WHERE salary > 70000) AS high_paid,
    AVG(salary) FILTER (WHERE hire_date > '2020-01-01') AS avg_newhire_salary
FROM employees
GROUP BY department;
```

### **Шпаргалка раздела 3**

#### **Базовый синтаксис GROUP BY:**

```sql
SELECT column1, AGG_FUNC(column2)
FROM table
WHERE condition
GROUP BY column1
HAVING condition_for_groups
ORDER BY column1
LIMIT N;
```

#### **Основные агрегатные функции:**
*   **`COUNT(*)`** — количество всех строк
*   **`COUNT(column)`** — количество не-NULL значений
*   **`SUM(column)`** — сумма значений
*   **`AVG(column)`** — среднее значение
*   **`MIN(column)`**, **`MAX(column)`** — минимальное/максимальное
*   **`GROUP_CONCAT()`** / **`STRING_AGG()`** — объединение строк

#### **Порядок выполнения:**
1.  **`FROM`** → 2. **`WHERE`** → 3. **`GROUP BY`** → 4. **`HAVING`** → 5. **`SELECT`** → 6. **`ORDER BY`** → 7. **`LIMIT`**

#### **Ключевые отличия WHERE vs HAVING:**
*   **`WHERE`** — фильтрация строк **до** группировки
*   **`HAVING`** — фильтрация групп **после** группировки
*   **`WHERE`** **не может** использовать агрегатные функции
*   **`HAVING`** **может** использовать агрегатные функции

#### **Типичные задачи и решения:**

```sql
-- 1. Подсчет уникальных значений
SELECT COUNT(DISTINCT column) FROM table;

-- 2. Нахождение дубликатов  
SELECT column, COUNT(*) FROM table GROUP BY column HAVING COUNT(*) > 1;

-- 3. Группировка по диапазонам
SELECT 
    CASE 
        WHEN salary < 40000 THEN 'Low'
        WHEN salary BETWEEN 40000 AND 80000 THEN 'Medium'
        ELSE 'High'
    END AS salary_group,
    COUNT(*) AS count
FROM employees
GROUP BY salary_group;

-- 4. Агрегация по периодам времени
SELECT 
    DATE_TRUNC('month', order_date) AS month,
    COUNT(*) AS orders_count,
    SUM(amount) AS total_amount
FROM orders
GROUP BY DATE_TRUNC('month', order_date)
ORDER BY month;
```

#### **Важные замечания:**
*   Все неагрегированные столбцы в `SELECT` должны быть в `GROUP BY`
*   Используйте `WHERE` для фильтрации строк, `HAVING` — для фильтрации групп
*   `COUNT(*)` быстрее `COUNT(column)` — не проверяет NULL
*   Для удаления дубликатов `GROUP BY` обычно эффективнее `DISTINCT`
*   Создавайте индексы на полях, используемых в `GROUP BY` и `WHERE`

## РАЗДЕЛ 4: СОЕДИНЕНИЯ ТАБЛИЦ (JOINs)

### **4.1. INNER JOIN**

**Что такое INNER JOIN?**
INNER JOIN возвращает только те строки, для которых нашлось соответствие **в обеих** соединяемых таблицах. Это самый распространенный тип соединения.

**Синтаксис:**
```sql
SELECT столбцы
FROM таблица1
INNER JOIN таблица2 
    ON таблица1.столбец = таблица2.столбец;
-- Ключевое слово INNER часто опускают
SELECT столбцы FROM таблица1 JOIN таблица2 ON условие;
```

**Диаграмма Венна:**
```
Таблица A      INNER JOIN      Таблица B
   (A)                         (B)
 ┌─────┐                     ┌─────┐
 │     │  Пересечение (A∩B)  │     │
 │  ┌──┼─────────────────────┼──┐  │
 │  │  │                     │  │  │
 └──┼──┘                     └──┼──┘
    │                           │
    └───────────────────────────┘
```

**Пример с данными:**

```sql
-- Таблицы для примеров
CREATE TABLE departments (
    id INT PRIMARY KEY,
    name VARCHAR(50)
);

CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    department_id INT,
    salary DECIMAL(10,2),
    FOREIGN KEY (department_id) REFERENCES departments(id)
);

-- INNER JOIN: сотрудники с привязанными отделами
SELECT 
    e.name AS employee_name,
    e.salary,
    d.name AS department_name
FROM employees e
INNER JOIN departments d ON e.department_id = d.id;

-- Результат (только сотрудники с department_id != NULL и существующим отделом):
-- | employee_name | salary | department_name |
-- |---------------|--------|-----------------|
-- | Иван Иванов   | 75000  | IT              |
-- | Анна Смирнова | 65000  | HR              |
-- | ...           | ...    | ...             |
```

**Особенности INNER JOIN:**
*   Порядок таблиц не важен для результата (но может влиять на производительность)
*   Если условие JOIN не выполняется, строка исключается из результата
*   Можно соединять несколько таблиц в цепочку

### **4.2. LEFT JOIN и RIGHT JOIN**

#### **LEFT (OUTER) JOIN**
Возвращает **все** строки из левой таблицы, даже если для них нет соответствия в правой таблице. Для отсутствующих соответствий возвращаются `NULL`.

```sql
-- Все сотрудники, даже без отделов
SELECT 
    e.name AS employee_name,
    e.salary,
    d.name AS department_name
FROM employees e
LEFT JOIN departments d ON e.department_id = d.id;

-- Результат:
-- | employee_name | salary | department_name |
-- |---------------|--------|-----------------|
-- | Иван Иванов   | 75000  | IT              |  -- Есть отдел
-- | Петр Петров   | 55000  | NULL            |  -- Отдела нет (NULL)
-- | Анна Смирнова | 65000  | HR              |  -- Есть отдел
```

#### **RIGHT (OUTER) JOIN**
Возвращает **все** строки из правой таблицы, даже если для них нет соответствия в левой. Используется реже, так как обычно можно поменять таблицы местами и использовать `LEFT JOIN`.

```sql
-- Все отделы, даже без сотрудников
SELECT 
    d.name AS department_name,
    e.name AS employee_name
FROM employees e
RIGHT JOIN departments d ON e.department_id = d.id;

-- Аналогично с LEFT JOIN (более читаемо):
SELECT 
    d.name AS department_name,
    e.name AS employee_name
FROM departments d
LEFT JOIN employees e ON d.id = e.department_id;
```

**Диаграммы Венна:**
```
LEFT JOIN:                     RIGHT JOIN:
Все из A + совпадения из B     Все из B + совпадения из A
     A                             A
  ┌─────┐                       ┌─────┐
  │     │                       │     │
  │  A∩B├───┐               ┌───┤ A∩B │
  │     │   │               │   │     │
  └─────┘   │               │   └─────┘
       │    │               │    │
       └────┼─────B         A────┼────┘
            │                     │
            B                     B
```

**Практическое использование LEFT JOIN:**

```sql
-- 1. Найти сотрудников без отдела
SELECT e.*
FROM employees e
LEFT JOIN departments d ON e.department_id = d.id
WHERE d.id IS NULL;

-- 2. Найти отделы без сотрудников
SELECT d.*
FROM departments d
LEFT JOIN employees e ON d.id = e.department_id
WHERE e.id IS NULL;

-- 3. Агрегация с учетом возможных отсутствующих связей
SELECT 
    d.name AS department_name,
    COUNT(e.id) AS employee_count,  -- COUNT(e.id), а не COUNT(*)
    COALESCE(AVG(e.salary), 0) AS avg_salary
FROM departments d
LEFT JOIN employees e ON d.id = e.department_id
GROUP BY d.id, d.name;
```

### **4.3. FULL OUTER JOIN**

**Что такое FULL OUTER JOIN?**
Возвращает **все строки из обеих таблиц**, объединяя их там, где есть соответствие. Если соответствия нет, для недостающих столбцов возвращается `NULL`.

```sql
-- Все сотрудники и все отделы
SELECT 
    e.name AS employee_name,
    d.name AS department_name
FROM employees e
FULL OUTER JOIN departments d ON e.department_id = d.id;

-- Результат включает:
-- 1. Сотрудников с отделами
-- 2. Сотрудников без отделов (department_name = NULL)
-- 3. Отделы без сотрудников (employee_name = NULL)
```

**Диаграмма Венна:**
```
FULL OUTER JOIN:
Все из A + все из B + пересечение
        A           B
     ┌─────┐     ┌─────┐
     │     │     │     │
     │  A∩B├─────┤ A∩B │
     │     │     │     │
     └─────┘     └─────┘
        │           │
        A только    B только
```

**Эмуляция FULL JOIN в MySQL:**
MySQL не поддерживает `FULL OUTER JOIN` напрямую, но его можно эмулировать:

```sql
-- Через UNION ALL
SELECT 
    e.name AS employee_name,
    d.name AS department_name
FROM employees e
LEFT JOIN departments d ON e.department_id = d.id

UNION ALL

SELECT 
    e.name AS employee_name,
    d.name AS department_name  
FROM employees e
RIGHT JOIN departments d ON e.department_id = d.id
WHERE e.department_id IS NULL;  -- Только строки, не попавшие в LEFT JOIN
```

### **4.4. CROSS JOIN**

**Что такое CROSS JOIN?**
Декартово произведение: каждая строка первой таблицы соединяется с каждой строкой второй таблицы. Результат = N × M строк.

```sql
-- Все комбинации размеров и цветов
SELECT 
    s.size_name,
    c.color_name
FROM sizes s
CROSS JOIN colors c;

-- То же самое через запятую (неявный CROSS JOIN)
SELECT s.size_name, c.color_name
FROM sizes s, colors c;

-- Практический пример: создание календаря
SELECT 
    months.month,
    years.year
FROM (VALUES ('Январь'), ('Февраль'), ('Март')) AS months(month)
CROSS JOIN (VALUES (2024), (2025), (2026)) AS years(year);
```

**Важные моменты:**
*   `CROSS JOIN` не имеет условия `ON`
*   Может генерировать огромное количество строк (10k × 10k = 100M)
*   Полезен для создания тестовых данных или комбинаций

### **4.5. SELF JOIN**

Соединение таблицы с самой собой для сравнения строк внутри одной таблицы.

```sql
-- Таблица сотрудников с менеджерами
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    manager_id INT,  -- Ссылается на id другого сотрудника
    salary DECIMAL(10,2)
);

-- Найти сотрудников и их менеджеров
SELECT 
    e.name AS employee_name,
    e.salary AS employee_salary,
    m.name AS manager_name,
    m.salary AS manager_salary
FROM employees e
LEFT JOIN employees m ON e.manager_id = m.id;  -- SELF JOIN

-- Найти сотрудников с зарплатой выше их менеджера
SELECT 
    e.name AS employee_name,
    e.salary,
    m.name AS manager_name,
    m.salary AS manager_salary
FROM employees e
JOIN employees m ON e.manager_id = m.id
WHERE e.salary > m.salary;
```

**Другие примеры SELF JOIN:**

```sql
-- Поиск пар сотрудников с одинаковой зарплатой
SELECT 
    e1.name AS employee1,
    e2.name AS employee2,
    e1.salary
FROM employees e1
JOIN employees e2 ON e1.salary = e2.salary 
    AND e1.id < e2.id  -- Чтобы избежать дубликатов (A-B и B-A)

-- Иерархические данные (подчиненность)
WITH RECURSIVE EmployeeHierarchy AS (
    SELECT id, name, manager_id, 1 AS level
    FROM employees WHERE manager_id IS NULL
    UNION ALL
    SELECT e.id, e.name, e.manager_id, eh.level + 1
    FROM employees e
    JOIN EmployeeHierarchy eh ON e.manager_id = eh.id
)
SELECT * FROM EmployeeHierarchy;
```


### **4.6. NATURAL JOIN и USING**

**NATURAL JOIN**
Автоматически соединяет таблицы по одинаковым именам столбцов. **Опасен и не рекомендуется к использованию.**

```sql
-- Опасный пример
SELECT * 
FROM employees 
NATURAL JOIN departments;
-- Соединит по ЛЮБОМУ столбцу с одинаковым именем (name, id, etc.)
```

**USING для простых случаев**
Если столбцы соединения имеют одинаковое имя в обеих таблицах:

```sql
-- Вместо ON table1.id = table2.id
SELECT *
FROM employees 
JOIN departments USING (department_id);

-- Эквивалентно:
SELECT *
FROM employees e
JOIN departments d ON e.department_id = d.department_id;
```



### **4.7. Неявные vs явные JOIN**

**Неявный JOIN (устаревший стиль):**

```sql
-- Старый синтаксис (из SQL-89)
SELECT e.name, d.name
FROM employees e, departments d
WHERE e.department_id = d.department_id
  AND e.salary > 50000;
```

**Явный JOIN (современный стиль):**

```sql
-- Современный синтаксис (SQL-92)
SELECT e.name, d.name
FROM employees e
JOIN departments d ON e.department_id = d.department_id
WHERE e.salary > 50000;
```

**Почему явный JOIN лучше:**
*   **Читаемость:** условие соединения отделено от условий фильтрации
*   **Безопасность:** меньше шансов забыть условие соединения (CROSS JOIN по ошибке)
*   **Поддержка OUTER JOIN:** старый синтаксис не поддерживает LEFT/RIGHT JOIN однозначно
*   **Стандарт:** современный стандарт SQL

### **Антипаттерны раздела 4**

#### ❌ **Забытое условие JOIN (случайный CROSS JOIN)**

```sql
-- ПЛОХО: Нет условия ON -> CROSS JOIN (декартово произведение)
SELECT * FROM employees JOIN departments;
-- Результат: каждый сотрудник × каждый отдел (огромный набор)

-- ✅ ХОРОШО: Всегда указывать условие JOIN
SELECT * FROM employees e 
JOIN departments d ON e.department_id = d.id;
```

#### ❌ **Использование SELECT * в JOIN**

```sql
-- ПЛОХО: Дублирующиеся имена столбцов (id, name в обеих таблицах)
SELECT * FROM employees e JOIN departments d ON e.department_id = d.id;

-- ✅ ХОРОШО: Явно перечислять нужные столбцы
SELECT 
    e.id AS employee_id,
    e.name AS employee_name,
    d.id AS department_id, 
    d.name AS department_name
FROM employees e 
JOIN departments d ON e.department_id = d.id;
```

#### ❌ **Неправильный тип JOIN**

```sql
-- ПЛОХО: INNER JOIN когда нужны все строки из одной таблицы
-- Потеря сотрудников без отделов
SELECT e.name, d.name 
FROM employees e 
JOIN departments d ON e.department_id = d.id;

-- ✅ ХОРОШО: Использовать LEFT JOIN для сохранения всех сотрудников
SELECT e.name, d.name 
FROM employees e 
LEFT JOIN departments d ON e.department_id = d.id;
```

#### ❌ **Множественные JOIN без планирования**

```sql
-- ПЛОХО: Соединение 4+ таблиц без фильтрации
SELECT * 
FROM orders o
JOIN customers c ON o.customer_id = c.id
JOIN products p ON o.product_id = p.id  
JOIN categories cat ON p.category_id = cat.id
JOIN suppliers s ON p.supplier_id = s.id;

-- ✅ ХОРОШО: Фильтровать на ранних этапах
SELECT * 
FROM orders o
JOIN customers c ON o.customer_id = c.id
JOIN products p ON o.product_id = p.id
WHERE o.order_date >= '2024-01-01'  -- Сначала отфильтровать
  AND c.country = 'RU';             -- Уменьшить объем данных для JOIN
```

### **Лучшие практики раздела 4**

#### ✅ **Использовать алиасы (псевдонимы) для таблиц**

```sql
-- Читаемо и компактно
SELECT 
    e.name AS emp_name,
    e.salary,
    d.name AS dept_name,
    m.name AS manager_name
FROM employees e
JOIN departments d ON e.department_id = d.id
LEFT JOIN employees m ON e.manager_id = m.id;
```

#### ✅ **Порядок таблиц в JOIN имеет значение для производительности**

```sql
-- ПЛОХО: Большая таблица первой
SELECT * FROM huge_table h 
JOIN small_table s ON h.id = s.huge_id;

-- ✅ ХОРОШО: Маленькую таблицу первой (часто)
SELECT * FROM small_table s 
JOIN huge_table h ON s.huge_id = h.id;
-- Оптимизатор может начать с меньшего набора данных
```

#### ✅ **Создавать индексы на полях соединения**

```sql
-- Индексы для ускорения JOIN
CREATE INDEX idx_employees_department ON employees(department_id);
CREATE INDEX idx_orders_customer ON orders(customer_id);
CREATE INDEX idx_orders_product ON orders(product_id);

-- Составные индексы для сложных условий
CREATE INDEX idx_performance ON orders(customer_id, order_date, status);
```

#### ✅ **Использовать EXPLAIN для анализа сложных JOIN**

```sql
-- Анализ плана выполнения
EXPLAIN 
SELECT e.name, d.name, p.name
FROM employees e
JOIN departments d ON e.department_id = d.id  
JOIN projects p ON e.id = p.lead_id;

-- В выводе смотрите:
-- 1. Тип JOIN (Nested Loop, Hash Join, Merge Join)
-- 2. Порядок таблиц
-- 3. Используются ли индексы
-- 4. Количество обрабатываемых строк
```

#### ✅ **Решать проблему N+1 запроса одним JOIN**

```sql
-- Проблема N+1 (в приложении):
-- 1. SELECT * FROM orders WHERE user_id = 5;  -- 10 заказов
-- 2. Для каждого заказа: SELECT * FROM products WHERE id = ? (10 запросов)

-- Решение: один запрос с JOIN
SELECT o.*, p.*
FROM orders o
JOIN products p ON o.product_id = p.id
WHERE o.user_id = 5;
```

### **Различия MySQL и PostgreSQL**

| Особенность | MySQL | PostgreSQL |
| :--- | :--- | :--- |
| **FULL OUTER JOIN** | Не поддерживается (эмулировать через UNION) | Полная поддержка |
| **Типы JOIN алгоритмов** | Nested Loop, Hash Join (с версии 8.0) | Nested Loop, Hash Join, Merge Join |
| **NATURAL JOIN** | Поддерживается (но опасен) | Поддерживается (но опасен) |
| **USING синтаксис** | Поддерживается | Поддерживается |
| **Производительность JOIN** | Зависит от движка (InnoDB лучше MyISAM) | Обычно лучше с большими данными |
| **Подсказки оптимизатору** | `STRAIGHT_JOIN` | `/*+ Leading(t1 t2) */` (через расширения) |

#### **Оптимизация JOIN в разных СУБД:**

```sql
-- MySQL: принудительный порядок JOIN
SELECT STRAIGHT_JOIN e.*, d.*
FROM employees e
JOIN departments d ON e.department_id = d.id;

-- PostgreSQL: статистика для оптимизатора
ANALYZE employees;
ANALYZE departments;
-- Затем оптимизатор выберет лучший план
```

### **Шпаргалка раздела 4**

#### **Синтаксис JOIN:**

```sql
SELECT столбцы
FROM таблица1
[INNER|LEFT|RIGHT|FULL] JOIN таблица2 
    ON условие_соединения
[WHERE условия_фильтрации]
[ORDER BY сортировка]
```

#### **Типы JOIN и их результаты:**

| Тип JOIN | Возвращает | Когда использовать |
| :--- | :--- | :--- |
| **INNER JOIN** | Только совпадающие строки из обеих таблиц | Связь обязательна (заказы-товары) |
| **LEFT JOIN** | Все строки слева + совпадения справа (NULL если нет) | Главная таблица + опциональные связи (сотрудники-отделы) |
| **RIGHT JOIN** | Все строки справа + совпадения слева (NULL если нет) | Редко, лучше поменять таблицы местами |
| **FULL JOIN** | Все строки из обеих таблиц | Когда нужны все данные из обеих таблиц |
| **CROSS JOIN** | Все комбинации (декартово произведение) | Комбинации (размеры-цвета), тестовые данные |

#### **Визуализация (диаграммы Венна):**

```
INNER:  A ∩ B      LEFT:   A ∪ (A ∩ B)    FULL:   A ∪ B
     ┌───┐              ┌───┐                  ┌───┐ ┌───┐
     │┼┼┼│              │┼┼┼│                  │┼┼┼│ │┼┼┼│
     └───┘              └───┘                  └───┘ └───┘
```

#### **Ключевые правила:**
*   Всегда указывайте условие JOIN (кроме CROSS JOIN)
*   Используйте алиасы для удобства чтения
*   Для опциональных связей используйте LEFT JOIN
*   Фильтруйте данные до JOIN, когда это возможно
*   Создавайте индексы на полях соединения
*   Избегайте NATURAL JOIN (опасно)
*   Для иерархических данных используйте SELF JOIN
*   При соединении 3+ таблиц анализируйте план выполнения (EXPLAIN)

#### **Примеры задач:**

```sql
-- Q: Как найти клиентов без заказов?
SELECT c.* 
FROM customers c
LEFT JOIN orders o ON c.id = o.customer_id
WHERE o.id IS NULL;

-- Q: Как найти менеджеров с наибольшим числом подчиненных?
SELECT m.name, COUNT(e.id) AS subordinates
FROM employees e
JOIN employees m ON e.manager_id = m.id
GROUP BY m.id, m.name
ORDER BY subordinates DESC
LIMIT 5;

-- Q: Как соединить 3 таблицы?
SELECT 
    o.order_date,
    c.name AS customer_name,
    p.name AS product_name,
    o.quantity * p.price AS total
FROM orders o
JOIN customers c ON o.customer_id = c.id
JOIN products p ON o.product_id = p.id
WHERE o.status = 'completed';
```

#### **Частые ошибки и решения:**
| Проблема | Решение |
| :--- | :--- |
| **Забыл условие JOIN → CROSS JOIN** | Всегда проверять наличие ON для INNER/LEFT/RIGHT JOIN |
| **Дубли имен столбцов в SELECT *** | Явно перечислять столбцы с алиасами |
| **Медленные JOIN на больших таблицах** | Индексы, фильтрация перед JOIN, анализ плана выполнения |
| **Не понимаю разницу между WHERE и ON** | WHERE фильтрует результат, ON определяет связь. Для LEFT JOIN разница критична! |

## РАЗДЕЛ 5: ПОДЗАПРОСЫ (SUBQUERIES)

### **5.1. Вложенные подзапросы**

**Что такое подзапрос?**
Подзапрос (вложенный запрос) — это SQL-запрос, вложенный в другой запрос. Он выполняется **первым**, и его результат используется внешним запросом.

**Где можно использовать подзапросы:**
*   В `SELECT` (скалярные подзапросы)
*   В `FROM` (табличные подзапросы)
*   В `WHERE` / `HAVING` (условные подзапросы)
*   В `INSERT`, `UPDATE`, `DELETE`

#### **Типы подзапросов по месту использования:**

**1. Скалярный подзапрос (в SELECT)**
Возвращает **одно значение** (одна строка, один столбец).

```sql
-- Добавить среднюю зарплату к каждой строке
SELECT 
    name,
    salary,
    (SELECT AVG(salary) FROM employees) AS avg_salary,
    salary - (SELECT AVG(salary) FROM employees) AS diff_from_avg
FROM employees;

-- Подзапрос в вычисляемом столбце
SELECT 
    order_id,
    quantity,
    price,
    quantity * price AS total,
    (SELECT AVG(quantity * price) FROM orders) AS avg_order_value
FROM orders;
```

**2. Табличный подзапрос (в FROM)**
Возвращает **таблицу** (много строк и столбцов).

```sql
-- Использовать результат подзапроса как временную таблицу
SELECT 
    dept_stats.department_name,
    dept_stats.employee_count,
    dept_stats.avg_salary
FROM (
    SELECT 
        d.name AS department_name,
        COUNT(e.id) AS employee_count,
        AVG(e.salary) AS avg_salary
    FROM departments d
    LEFT JOIN employees e ON d.id = e.department_id
    GROUP BY d.id, d.name
) AS dept_stats
WHERE dept_stats.employee_count > 5
ORDER BY dept_stats.avg_salary DESC;

-- Подзапрос с несколькими столбцами
SELECT 
    region,
    total_sales,
    total_sales * 100.0 / (SELECT SUM(amount) FROM sales) AS percentage
FROM (
    SELECT 
        region,
        SUM(amount) AS total_sales
    FROM sales
    GROUP BY region
) AS regional_sales;
```

**3. Подзапрос в WHERE**

```sql
-- Найти сотрудников с зарплатой выше средней
SELECT name, salary
FROM employees
WHERE salary > (SELECT AVG(salary) FROM employees);

-- Сравнение с агрегированным значением
SELECT *
FROM products
WHERE price > (SELECT AVG(price) FROM products WHERE category = 'Электроника');
```

### **5.2. Коррелированные подзапросы**

#### **Что такое коррелированный подзапрос?**
Подзапрос, который ссылается на столбцы внешнего запроса. Выполняется один раз для каждой строки внешнего запроса.

```sql
-- Найти сотрудников с зарплатой выше средней по их отделу
SELECT 
    e1.name,
    e1.salary,
    e1.department_id
FROM employees e1
WHERE salary > (
    SELECT AVG(salary)
    FROM employees e2
    WHERE e2.department_id = e1.department_id  -- Ссылка на внешний запрос!
);

-- Еще пример: последний заказ каждого клиента
SELECT 
    c.name,
    o.order_date,
    o.amount
FROM customers c
JOIN orders o ON c.id = o.customer_id
WHERE o.order_date = (
    SELECT MAX(order_date)
    FROM orders o2
    WHERE o2.customer_id = c.id  -- Корреляция
);
```

#### **Ключевые особенности коррелированных подзапросов:**
*   Выполняются для каждой строки внешнего запроса
*   Могут быть медленными на больших таблицах
*   Часто можно заменить на JOIN


### **5.3. Подзапросы в SELECT/WHERE/FROM**

#### **Подзапросы в SELECT (скалярные)**

```sql
-- Для каждого отдела добавить количество сотрудников и среднюю зарплату
SELECT 
    d.name AS department,
    (SELECT COUNT(*) FROM employees e WHERE e.department_id = d.id) AS emp_count,
    (SELECT AVG(salary) FROM employees e WHERE e.department_id = d.id) AS avg_salary
FROM departments d
ORDER BY emp_count DESC;
```

#### **Подзапросы в WHERE с операторами сравнения**

```sql
-- Операторы: =, >, <, >=, <=, <>
-- Важно: подзапрос должен возвращать одно значение!

-- Найти самый дорогой товар
SELECT * FROM products
WHERE price = (SELECT MAX(price) FROM products);

-- Товары дороже среднего в своей категории
SELECT *
FROM products p1
WHERE price > (
    SELECT AVG(price)
    FROM products p2
    WHERE p2.category = p1.category
);
```

#### **Подзапросы в WHERE с IN / NOT IN**

```sql
-- IN: значение совпадает с одним из списка
-- Найти клиентов, которые делали заказы
SELECT *
FROM customers
WHERE id IN (SELECT DISTINCT customer_id FROM orders);

-- NOT IN: обратное
-- Найти клиентов без заказов
SELECT *
FROM customers
WHERE id NOT IN (SELECT DISTINCT customer_id FROM orders);
```

#### **Подзапросы в WHERE с EXISTS / NOT EXISTS**

```sql
-- EXISTS: проверяет существование строк
-- Клиенты с заказы
SELECT *
FROM customers c
WHERE EXISTS (
    SELECT 1 FROM orders o WHERE o.customer_id = c.id
);

-- NOT EXISTS: обратное
-- Клиенты без заказов
SELECT *
FROM customers c
WHERE NOT EXISTS (
    SELECT 1 FROM orders o WHERE o.customer_id = c.id
);

-- EXISTS vs IN:
-- EXISTS обычно быстрее для коррелированных подзапросов
-- IN может быть быстрее для небольших не-коррелированных подзапросов
```


### **5.4. EXISTS vs IN vs JOIN**

#### **Сравнение трех подходов для одной задачи:**
Задача: найти клиентов с заказами.

**1. С помощью EXISTS:**
```sql
SELECT *
FROM customers c
WHERE EXISTS (
    SELECT 1 FROM orders o WHERE o.customer_id = c.id
);
```
*Плюсы:* Обычно оптимально для коррелированных запросов  
*Минусы:* Менее интуитивно для простых случаев

**2. С помощью IN:**
```sql
SELECT *
FROM customers
WHERE id IN (SELECT customer_id FROM orders);
```
*Плюсы:* Читаемо для простых случаев  
*Минусы:* Медленно, если подзапрос возвращает много значений  
*Осторожно:* NULL значения могут привести к неожиданным результатам

**3. С помощью JOIN:**
```sql
SELECT DISTINCT c.*
FROM customers c
JOIN orders o ON c.id = o.customer_id;
```
*Плюсы:* Часто самый быстрый вариант  
*Минусы:* Может возвращать дубликаты (нужен DISTINCT)  
*Лучше всего:* Для простых связей и когда нужны данные из обеих таблиц

#### **Производительность:**
*   Для простых случаев: **JOIN** (с DISTINCT если нужно) или **EXISTS**
*   Для коррелированных запросов: **EXISTS**
*   Для небольших статических списков: **IN**

#### **NULL проблема с NOT IN:**
```sql
-- ОПАСНО: Если подзапрос возвращает NULL, NOT IN вернет пустой результат
SELECT * FROM table1 
WHERE id NOT IN (SELECT id FROM table2 WHERE condition);
-- Если table2 содержит NULL в id, запрос вернет 0 строк

-- БЕЗОПАСНО:
SELECT * FROM table1 
WHERE id NOT IN (
    SELECT id FROM table2 
    WHERE condition AND id IS NOT NULL
);

-- Или используйте NOT EXISTS (безопасно с NULL)
SELECT * FROM table1 t1
WHERE NOT EXISTS (
    SELECT 1 FROM table2 t2 
    WHERE t2.id = t1.id
);
```

### **Антипаттерны раздела 5**

#### ❌ **Избыточные вложенные подзапросы**
```sql
-- ПЛОХО: Несколько одинаковых подзапросов
SELECT 
    name,
    salary,
    (SELECT AVG(salary) FROM employees) AS avg_salary,
    salary - (SELECT AVG(salary) FROM employees) AS diff
FROM employees;

-- ✅ ХОРОШО: Вычислить один раз с помощью CTE или переменной
WITH avg_salary AS (
    SELECT AVG(salary) AS avg_val FROM employees
)
SELECT 
    name,
    salary,
    avg_salary.avg_val,
    salary - avg_salary.avg_val AS diff
FROM employees, avg_salary;
```

#### ❌ **Подзапрос в SELECT, возвращающий несколько строк**
```sql
-- ПЛОХО: Ошибка, если подзапрос вернет больше одной строки
SELECT 
    name,
    (SELECT order_date FROM orders WHERE customer_id = c.id) AS last_order
FROM customers c;

-- ✅ ХОРОШО: Использовать агрегатную функцию или LIMIT
SELECT 
    name,
    (SELECT MAX(order_date) FROM orders WHERE customer_id = c.id) AS last_order
FROM customers c;

-- ИЛИ использовать JOIN
SELECT 
    c.name,
    MAX(o.order_date) AS last_order
FROM customers c
LEFT JOIN orders o ON c.id = o.customer_id
GROUP BY c.id, c.name;
```

#### ❌ **Коррелированный подзапрос на большой таблице**
```sql
-- ПЛОХО: 100k строк × подзапрос для каждой строки
SELECT 
    e1.name,
    (SELECT COUNT(*) FROM orders o WHERE o.employee_id = e1.id) AS order_count
FROM employees e1;

-- ✅ ХОРОШО: Использовать JOIN с группировкой
SELECT 
    e.name,
    COUNT(o.id) AS order_count
FROM employees e
LEFT JOIN orders o ON e.id = o.employee_id
GROUP BY e.id, e.name;

-- ИЛИ использовать оконную функцию
SELECT DISTINCT
    e.name,
    COUNT(o.id) OVER (PARTITION BY e.id) AS order_count
FROM employees e
LEFT JOIN orders o ON e.id = o.employee_id;
```

#### ❌ **Подзапросы вместо простых условий**
```sql
-- ПЛОХО: Избыточный подзапрос
SELECT * FROM products
WHERE category IN (SELECT 'Электроника' UNION SELECT 'Бытовая техника');

-- ✅ ХОРОШО: Простое условие
SELECT * FROM products
WHERE category IN ('Электроника', 'Бытовая техника');
```

### **Лучшие практики раздела 5**

#### ✅ **Использовать CTE для сложных подзапросов**
```sql
-- Вместо глубокой вложенности
SELECT * FROM (
    SELECT * FROM (
        SELECT * FROM table1 WHERE condition
    ) t1 JOIN table2 ON ...
) t2 WHERE ...

-- ✅ Читаемо с CTE
WITH filtered_table1 AS (
    SELECT * FROM table1 WHERE condition
),
joined_data AS (
    SELECT * FROM filtered_table1 t1
    JOIN table2 ON t1.id = table2.t1_id
)
SELECT * FROM joined_data WHERE final_condition;
```

#### ✅ **Выносить общие подзапросы в CTE**
```sql
-- Повторяющийся подзапрос
SELECT 
    (SELECT AVG(price) FROM products) AS avg_price,
    p.name,
    p.price - (SELECT AVG(price) FROM products) AS price_diff
FROM products p;

-- ✅ Вынести в CTE
WITH avg_price AS (
    SELECT AVG(price) AS avg_val FROM products
)
SELECT 
    avg_price.avg_val,
    p.name,
    p.price - avg_price.avg_val AS price_diff
FROM products p, avg_price;
```

#### ✅ **Проверять количество строк в подзапросах**
```sql
-- Всегда убеждайтесь, что подзапрос возвращает ожидаемое количество строк
-- Для условий =, >, < и т.д. - должен возвращать 1 строку
-- Для IN, EXISTS - может возвращать любое количество

-- Проверить перед использованием
SELECT COUNT(*) FROM (
    SELECT customer_id FROM orders WHERE total > 1000
) sub;

-- Использовать LIMIT 1 если нужна одна строка
SELECT *
FROM employees
WHERE salary = (
    SELECT salary FROM employees 
    ORDER BY salary DESC 
    LIMIT 1 OFFSET 1  -- Вторая по величине зарплата
);
```

#### ✅ **Оптимизировать коррелированные подзапросы**
```sql
-- Оригинальный медленный коррелированный подзапрос
SELECT 
    c.name,
    (SELECT COUNT(*) FROM orders o WHERE o.customer_id = c.id) AS order_count
FROM customers c;

-- ✅ Оптимизация 1: LEFT JOIN с группировкой
SELECT 
    c.name,
    COUNT(o.id) AS order_count
FROM customers c
LEFT JOIN orders o ON c.id = o.customer_id
GROUP BY c.id, c.name;

-- ✅ Оптимизация 2: Предварительная агрегация
WITH order_counts AS (
    SELECT customer_id, COUNT(*) AS cnt
    FROM orders
    GROUP BY customer_id
)
SELECT 
    c.name,
    COALESCE(oc.cnt, 0) AS order_count
FROM customers c
LEFT JOIN order_counts oc ON c.id = oc.customer_id;
```


### **Различия MySQL и PostgreSQL**

| Особенность | MySQL | PostgreSQL |
| :--- | :--- | :--- |
| **Поддержка LIMIT в подзапросах** | Да | Да |
| **Поддержка ORDER BY в подзапросах** | Да, но часто игнорируется если не с LIMIT | Да |
| **Коррелированные подзапросы в SELECT** | Поддерживаются | Поддерживаются |
| **Производительность подзапросов** | Может быть ниже чем JOIN | Часто хорошо оптимизируются |
| **Поддержка LATERAL** | С версии 8.0.14 | Полная поддержка |
| **Подзапросы в VALUES** | Да | Да |

#### **LATERAL подзапросы (PostgreSQL и MySQL 8.0+):**
```sql
-- LATERAL позволяет подзапросу ссылаться на столбцы предыдущих таблиц
SELECT 
    d.name AS department,
    top_emp.name,
    top_emp.salary
FROM departments d,
LATERAL (
    SELECT name, salary
    FROM employees e
    WHERE e.department_id = d.id
    ORDER BY salary DESC
    LIMIT 3
) top_emp;

-- Эквивалент в старом стиле (с коррелированным подзапросом)
SELECT 
    d.name AS department,
    e.name,
    e.salary
FROM departments d
JOIN employees e ON e.department_id = d.id
WHERE e.salary IN (
    SELECT salary 
    FROM employees e2 
    WHERE e2.department_id = d.id 
    ORDER BY salary DESC 
    LIMIT 3
);
```

### **Шпаргалка раздела 5**

#### **Синтаксис подзапросов:**
```sql
-- В SELECT (скалярный)
SELECT col1, (SELECT ...) FROM table;

-- В FROM (табличный)
SELECT * FROM (SELECT ...) AS subquery;

-- В WHERE
SELECT * FROM table WHERE col OPERATOR (SELECT ...);

-- В EXISTS
SELECT * FROM table WHERE EXISTS (SELECT ...);
```

#### **Типы подзапросов:**
*   **Скалярный:** одна строка, один столбец → в SELECT, WHERE с =
*   **Строковый:** одна строка, несколько столбцов → редко
*   **Табличный:** много строк, много столбцов → в FROM
*   **Коррелированный:** ссылается на внешний запрос → может быть медленным

#### **Операторы для подзапросов в WHERE:**
*   `= > < >= <= <>` — подзапрос должен вернуть 1 значение
*   `IN` / `NOT IN` — для сравнения с набором значений
*   `EXISTS` / `NOT EXISTS` — проверка существования
*   `ANY` / `SOME` — сравнение с любым значением из набора
*   `ALL` — сравнение со всеми значениями из набора

#### **Примеры задач:**
```sql
-- 1. Найти второй по величине оклад
SELECT MAX(salary) 
FROM employees 
WHERE salary < (SELECT MAX(salary) FROM employees);

-- 2. Найти отделы без сотрудников
SELECT * FROM departments d
WHERE NOT EXISTS (
    SELECT 1 FROM employees e WHERE e.department_id = d.id
);

-- 3. Найти сотрудников, зарабатывающих больше среднего по компании
SELECT name, salary 
FROM employees 
WHERE salary > (SELECT AVG(salary) FROM employees);

-- 4. Найти товары, которые никогда не заказывались
SELECT * FROM products p
WHERE NOT EXISTS (
    SELECT 1 FROM order_items oi WHERE oi.product_id = p.id
);

-- 5. Найти менеджеров, у которых больше 5 подчиненных
SELECT m.name, COUNT(e.id) AS subordinates
FROM employees e
JOIN employees m ON e.manager_id = m.id
GROUP BY m.id, m.name
HAVING COUNT(e.id) > 5;
```

#### **Ключевые правила:**
*   Подзапрос в SELECT должен возвращать одно значение
*   Подзапрос в FROM должен иметь алиас
*   Коррелированные подзапросы выполняются для каждой строки → могут быть медленными
*   EXISTS обычно эффективнее IN для коррелированных подзапросов
*   IN имеет проблемы с NULL значениями
*   Часто подзапросы можно заменить на JOIN (обычно эффективнее)
*   Используйте CTE для сложных вложенных запросов
*   Всегда проверяйте план выполнения для подзапросов

#### **Когда использовать подзапросы vs JOIN:**
*   **Используйте подзапросы, когда:**
    *   Нужно одно агрегированное значение в SELECT
    *   Проверка существования (EXISTS / NOT EXISTS)
    *   Сложная логика, которую проще выразить подзапросом
*   **Используйте JOIN, когда:**
    *   Нужны данные из нескольких таблиц
    *   Можно избежать коррелированных подзапросов
    *   Запрос должен быть максимально производительным

#### **Производительность подзапросов:**
*   Скалярные подзапросы в SELECT → могут быть медленными если много строк
*   Коррелированные подзапросы → O(n²) сложность
*   EXISTS → обычно хорошо оптимизируется
*   IN с большим списком → может быть медленно
*   Табличные подзапросы в FROM → обычно хорошо, если есть индексы

## РАЗДЕЛ 6: CTE (COMMON TABLE EXPRESSIONS)

### **6.1. Простые CTE**

**Что такое CTE?**
CTE (Common Table Expression, обобщенное табличное выражение) — это временный результат запроса, который можно использовать в основном запросе. По сути, это именованная временная таблица, существующая только во время выполнения запроса.

**Синтаксис:**
```sql
WITH имя_cte AS (
    SELECT ...  -- Подзапрос
)
SELECT * FROM имя_cte;
```

**Зачем использовать CTE?**
*   **Улучшение читаемости:** Разбивает сложные запросы на логические блоки
*   **Переиспользование:** Можно ссылаться на CTE несколько раз в запросе
*   **Рекурсивные запросы:** Единственный способ выполнить рекурсию в SQL
*   **Отладка:** Легко тестировать отдельные части запроса
*   **Замена представлений:** Когда нужен одноразовый "view"

**Базовый пример:**

```sql
-- Создаем CTE с сотрудниками IT отдела
WITH it_employees AS (
    SELECT id, name, salary
    FROM employees
    WHERE department = 'IT'
)
-- Используем CTE в основном запросе
SELECT 
    name,
    salary,
    salary * 0.87 AS salary_after_tax
FROM it_employees
WHERE salary > 70000
ORDER BY salary DESC;
```

**Практический пример — подготовка данных:**

```sql
-- CTE для фильтрации и преобразования данных перед анализом
WITH cleaned_sales AS (
    SELECT 
        order_id,
        customer_id,
        product_id,
        -- Убираем выбросы
        CASE 
            WHEN amount < 0 THEN 0
            WHEN amount > 100000 THEN 100000
            ELSE amount
        END AS cleaned_amount,
        order_date
    FROM sales
    WHERE status = 'completed'
      AND order_date >= '2024-01-01'
),
customer_totals AS (
    SELECT 
        customer_id,
        COUNT(*) AS order_count,
        SUM(cleaned_amount) AS total_spent
    FROM cleaned_sales
    GROUP BY customer_id
)
-- Основной запрос с использованием обоих CTE
SELECT 
    c.name AS customer_name,
    ct.order_count,
    ct.total_spent,
    ROUND(ct.total_spent / ct.order_count, 2) AS avg_order_value
FROM customer_totals ct
JOIN customers c ON ct.customer_id = c.id
WHERE ct.order_count >= 3
ORDER BY ct.total_spent DESC;
```

### **6.2. Многоуровневые CTE**

**Несколько CTE в одном запросе:**

```sql
WITH 
-- Первое CTE: активные пользователи
active_users AS (
    SELECT 
        user_id,
        MIN(signup_date) AS first_seen,
        MAX(last_login) AS last_seen
    FROM user_sessions
    WHERE last_login >= CURRENT_DATE - INTERVAL '30 days'
    GROUP BY user_id
),

-- Второе CTE: пользовательские заказы
user_orders AS (
    SELECT 
        user_id,
        COUNT(*) AS order_count,
        SUM(amount) AS total_spent
    FROM orders
    WHERE order_date >= CURRENT_DATE - INTERVAL '90 days'
    GROUP BY user_id
),

-- Третье CTE: комбинированная статистика
user_stats AS (
    SELECT 
        au.user_id,
        au.first_seen,
        au.last_seen,
        COALESCE(uo.order_count, 0) AS order_count,
        COALESCE(uo.total_spent, 0) AS total_spent,
        CASE 
            WHEN uo.order_count >= 5 THEN 'VIP'
            WHEN uo.order_count >= 2 THEN 'Активный'
            ELSE 'Новый'
        END AS user_segment
    FROM active_users au
    LEFT JOIN user_orders uo ON au.user_id = uo.user_id
)

-- Основной запрос
SELECT 
    u.name,
    us.first_seen,
    us.last_seen,
    us.order_count,
    us.total_spent,
    us.user_segment,
    DATEDIFF(CURRENT_DATE, us.last_seen) AS days_since_last_activity
FROM user_stats us
JOIN users u ON us.user_id = u.id
WHERE us.user_segment = 'VIP'
ORDER BY us.total_spent DESC;
```

**Цепочка зависимостей:**

```sql
-- Каждое последующее CTE может ссылаться на предыдущие
WITH 
step1 AS (SELECT ... FROM source_table),
step2 AS (SELECT ... FROM step1 WHERE ...),
step3 AS (SELECT ... FROM step2 JOIN other_table ...)
SELECT * FROM step3 WHERE ...;
```


### **6.3. Рекурсивные CTE**

#### **Что такое рекурсивные CTE?**
CTE, которые могут ссылаться на самих себя, позволяя обрабатывать иерархические или рекурсивные структуры данных.

#### **Структура рекурсивного CTE:**

```sql
WITH RECURSIVE имя_cte AS (
    -- Базовый случай (anchor member)
    SELECT ... FROM таблица WHERE условие_начала
    
    UNION ALL
    
    -- Рекурсивный случай (recursive member)
    SELECT ... FROM имя_cte JOIN таблица ON условие_рекурсии
    WHERE условие_останова
)
SELECT * FROM имя_cte;
```

#### **Пример 1: Иерархия сотрудников (от менеджера к подчиненным)**

```sql
-- Таблица сотрудников с менеджерами
CREATE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    manager_id INT,
    FOREIGN KEY (manager_id) REFERENCES employees(id)
);

-- Рекурсивный CTE для построения иерархии
WITH RECURSIVE employee_hierarchy AS (
    -- Базовый случай: верхнеуровневые менеджеры (у которых нет менеджера)
    SELECT 
        id,
        name,
        manager_id,
        1 AS level,
        name::TEXT AS path  -- Путь от корня
    FROM employees
    WHERE manager_id IS NULL
    
    UNION ALL
    
    -- Рекурсивный случай: находим подчиненных
    SELECT 
        e.id,
        e.name,
        e.manager_id,
        eh.level + 1 AS level,
        eh.path || ' -> ' || e.name AS path
    FROM employees e
    JOIN employee_hierarchy eh ON e.manager_id = eh.id
)
SELECT 
    id,
    name,
    level,
    path,
    -- Отступ для визуализации иерархии
    LPAD('', (level - 1) * 4, ' ') || name AS hierarchy_view
FROM employee_hierarchy
ORDER BY path;

-- Результат:
-- | id | name     | level | path                    | hierarchy_view  |
-- |----|----------|-------|-------------------------|-----------------|
-- | 1  | Иван     | 1     | Иван                    | Иван            |
-- | 2  | Анна     | 2     | Иван -> Анна            |     Анна        |
-- | 3  | Петр     | 2     | Иван -> Петр            |     Петр        |
-- | 4  | Мария    | 3     | Иван -> Петр -> Мария   |         Мария   |
```

#### **Пример 2: Генерация последовательности дат**

```sql
-- Генерация всех дат за последние 30 дней
WITH RECURSIVE date_series AS (
    -- Начинаем с сегодняшней даты
    SELECT CURRENT_DATE AS date
    
    UNION ALL
    
    -- Отнимаем по одному дню
    SELECT date - INTERVAL '1 day'
    FROM date_series
    WHERE date > CURRENT_DATE - INTERVAL '29 days'  -- Останавливаемся через 30 итераций
)
SELECT 
    date,
    EXTRACT(DOW FROM date) AS day_of_week,
    TO_CHAR(date, 'Day') AS day_name
FROM date_series
ORDER BY date;

-- Пример с заполнением пропусков в данных
WITH RECURSIVE all_dates AS (
    SELECT DATE('2024-01-01') AS date
    UNION ALL
    SELECT date + INTERVAL '1 day'
    FROM all_dates
    WHERE date < '2024-01-31'
),
daily_sales AS (
    SELECT 
        sale_date,
        SUM(amount) AS total_sales
    FROM sales
    WHERE sale_date BETWEEN '2024-01-01' AND '2024-01-31'
    GROUP BY sale_date
)
SELECT 
    ad.date,
    COALESCE(ds.total_sales, 0) AS total_sales,
    SUM(COALESCE(ds.total_sales, 0)) OVER (ORDER BY ad.date) AS cumulative_sales
FROM all_dates ad
LEFT JOIN daily_sales ds ON ad.date = ds.sale_date
ORDER BY ad.date;
```

#### **Пример 3: Поиск в графах (пути между узлами)**

```sql
-- Таблица связей между узлами
CREATE TABLE graph_edges (
    from_node INT,
    to_node INT,
    distance INT
);

-- Найти все пути от узла 1 к узлу 5
WITH RECURSIVE graph_paths AS (
    -- Базовый случай: начинаем с узла 1
    SELECT 
        from_node AS start_node,
        to_node AS end_node,
        distance,
        ARRAY[from_node, to_node] AS path,
        FALSE AS cycle  -- Отслеживаем циклы
    FROM graph_edges
    WHERE from_node = 1
    
    UNION ALL
    
    -- Рекурсивный случай: продлеваем путь
    SELECT 
        gp.start_node,
        ge.to_node,
        gp.distance + ge.distance,
        gp.path || ge.to_node,
        ge.to_node = ANY(gp.path)  -- Проверяем, не посещали ли уже этот узел
    FROM graph_paths gp
    JOIN graph_edges ge ON gp.end_node = ge.from_node
    WHERE NOT gp.cycle  -- Останавливаемся если обнаружили цикл
      AND array_length(gp.path, 1) < 10  -- Ограничиваем глубину рекурсии
)
SELECT 
    start_node,
    end_node,
    distance,
    path
FROM graph_paths
WHERE end_node = 5  -- Ищем пути к узлу 5
ORDER BY distance, array_length(path, 1);

-- Результат покажет все возможные пути от 1 к 5 с общей длиной
```

### **6.4. CTE vs подзапросы vs временные таблицы**

#### **Сравнение подходов:**

| Критерий | CTE | Подзапросы | Временные таблицы |
| :--- | :--- | :--- | :--- |
| **Читаемость** | ✅ Высокая (логические блоки) | ❌ Низкая (вложенность) | ✅ Средняя |
| **Переиспользование** | ✅ В пределах одного запроса | ❌ Нет | ✅ В пределах сессии |
| **Производительность** | Зависит от СУБД | Часто хуже CTE | ✅ Хорошая (индексы, статистика) |
| **Объем данных** | Ограничен памятью | Ограничен памятью | ✅ Большие объемы |
| **Сложность логики** | ✅ Высокая (рекурсия) | Средняя | ✅ Высокая |
| **Отладка** | ✅ Легко тестировать части | ❌ Сложно | ✅ Легко |
| **Длительность** | Только на время запроса | Только на время запроса | ✅ На время сессии |

#### **Когда что использовать:**

**Используйте CTE, когда:**
*   Нужна рекурсивная обработка данных
*   Запрос сложный и нужно улучшить читаемость
*   Нужно несколько раз сослаться на один результат
*   Вы работаете с небольшими или средними объемами данных

```sql
-- Пример где CTE лучше подзапроса
-- ❌ Сложно читать
SELECT * FROM (
    SELECT * FROM (
        SELECT * FROM table1 WHERE condition1
    ) t1 JOIN table2 ON t1.id = table2.id
) t2 WHERE condition2;

-- ✅ Читаемо с CTE
WITH 
filtered_t1 AS (SELECT * FROM table1 WHERE condition1),
joined_data AS (SELECT * FROM filtered_t1 JOIN table2 ON id = table2.id)
SELECT * FROM joined_data WHERE condition2;
```

**Используйте временные таблицы, когда:**
*   Очень большие объемы данных
*   Нужны индексы для производительности
*   Результат используется в нескольких запросах
*   Нужно сохранить промежуточные результаты для отладки

```sql
-- Временная таблица в PostgreSQL
CREATE TEMP TABLE temp_user_stats AS
SELECT 
    user_id,
    COUNT(*) AS order_count,
    SUM(amount) AS total_spent
FROM orders
GROUP BY user_id;

-- Создаем индекс для производительности
CREATE INDEX idx_temp_user ON temp_user_stats(user_id);

-- Используем в нескольких запросах
SELECT * FROM temp_user_stats WHERE order_count > 10;

-- Очищаем вручную или при закрытии сессии
DROP TABLE temp_user_stats;
```

**Используйте подзапросы, когда:**
*   Простая логика, не требующая CTE
*   Нужно одно значение в SELECT или WHERE
*   Работа с EXISTS/NOT EXISTS
*   Не хотите вводить дополнительную сложность

### **6.5. Материализация CTE**

#### **Что такое материализация CTE?**
Некоторые СУБД позволяют явно указать, что CTE должен быть материализован — то есть, его результат вычисляется один раз и сохраняется во временной таблице.

#### **PostgreSQL:**

```sql
-- Автоматическая материализация
WITH large_cte AS MATERIALIZED (
    SELECT * FROM huge_table WHERE complex_condition()
)
SELECT * FROM large_cte c1
JOIN large_cte c2 ON c1.id = c2.parent_id;

-- NOT MATERIALIZED (принудительно не материализовать)
WITH small_cte AS NOT MATERIALIZED (
    SELECT * FROM small_table WHERE simple_condition()
)
SELECT * FROM small_cte;

-- Оптимизация: материализуем дорогой CTE
WITH 
expensive_calc AS MATERIALIZED (
    -- Дорогие вычисления
    SELECT 
        user_id,
        EXP(SUM(LN(value))) AS geometric_mean,  -- Дорогая операция
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY value) AS median
    FROM measurements
    GROUP BY user_id
),
light_processing AS NOT MATERIALIZED (
    -- Легкие операции
    SELECT user_id, name FROM users WHERE active = true
)
SELECT 
    lp.name,
    ec.geometric_mean,
    ec.median
FROM light_processing lp
JOIN expensive_calc ec ON lp.user_id = ec.user_id;
```

#### **MySQL 8.0+:**
```sql
-- MySQL автоматически решает материализовать CTE или нет
-- Можно влиять через оптимизатор
WITH cte AS (
    SELECT /*+ MERGE() */ * FROM t1  -- Попробовать не материализовать
    -- SELECT /*+ MATERIALIZE() */ * FROM t1  -- Попробовать материализовать
)
SELECT * FROM cte;
```

#### **Преимущества материализации:**
*   **Производительность:** Дорогой CTE вычисляется один раз
*   **Стабильность:** Гарантированный план выполнения
*   **Память:** Можно сбросить в tempdb при больших данных

#### **Недостатки:**
*   Дополнительные накладные расходы на создание временной таблицы
*   Потеря индексов исходных таблиц
*   Может быть излишним для маленьких CTE

#### **Когда материализовать CTE:**

```sql
-- ❌ НЕ НАДО: CTE маленький и используется 1 раз
WITH small_data AS (
    SELECT id, name FROM users WHERE id = 123
)
SELECT * FROM small_data;

-- ✅ НАДО: CTE большой и используется несколько раз
WITH big_data AS MATERIALIZED (
    SELECT * FROM huge_table 
    WHERE date BETWEEN '2024-01-01' AND '2024-12-31'
    -- сложные вычисления
)
SELECT * FROM big_data b1
JOIN big_data b2 ON b1.category = b2.category
WHERE b1.amount > b2.amount;
```

### **Антипаттерны раздела 6**

#### ❌ **CTE без необходимости (простой запрос)**

```sql
-- ПЛОХО: CTE для простого запроса добавляет ненужную сложность
WITH users AS (SELECT * FROM user_table)
SELECT name FROM users WHERE active = true;

-- ✅ ХОРОШО: Просто запрос
SELECT name FROM user_table WHERE active = true;
```

#### ❌ **Бесконечная рекурсия**

```sql
-- ОПАСНО: Нет условия остановки
WITH RECURSIVE infinite AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM infinite  -- Бесконечное увеличение!
)
SELECT * FROM infinite;

-- ✅ БЕЗОПАСНО: Добавить условие остановки
WITH RECURSIVE limited AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM limited
    WHERE n < 100  -- Ограничение
)
SELECT * FROM limited;
```

#### ❌ **CTE для изменения данных (зависит от СУБД)**

```sql
-- ПЛОХО: В некоторых СУБД CTE для UPDATE/DELETE может вести себя неожиданно
WITH to_delete AS (
    SELECT id FROM logs WHERE created_at < '2023-01-01'
)
DELETE FROM logs WHERE id IN (SELECT id FROM to_delete);

-- ✅ ХОРОШО: Использовать подзапрос или временную таблицу
DELETE FROM logs 
WHERE created_at < '2023-01-01';

-- Или явная временная таблица
CREATE TEMP TABLE ids_to_delete AS 
SELECT id FROM logs WHERE created_at < '2023-01-01';
DELETE FROM logs WHERE id IN (SELECT id FROM ids_to_delete);
```

#### ❌ **Игнорирование ограничений рекурсии**

```sql
-- ПЛОХО: Может превысить лимит глубины рекурсии
WITH RECURSIVE deep_tree AS (
    SELECT id, parent_id, 1 AS depth FROM tree WHERE parent_id IS NULL
    UNION ALL
    SELECT t.id, t.parent_id, dt.depth + 1
    FROM tree t
    JOIN deep_tree dt ON t.parent_id = dt.id
)
SELECT * FROM deep_tree;  -- Может быть очень глубоким!

-- ✅ ХОРОШО: Добавить ограничение глубины
WITH RECURSIVE safe_tree AS (
    SELECT id, parent_id, 1 AS depth FROM tree WHERE parent_id IS NULL
    UNION ALL
    SELECT t.id, t.parent_id, st.depth + 1
    FROM tree t
    JOIN safe_tree st ON t.parent_id = st.id
    WHERE st.depth < 50  -- Ограничение глубины
)
SELECT * FROM safe_tree;
```

### **Лучшие практики раздела 6**

#### ✅ **Использовать CTE для сложных бизнес-логик**

```sql
-- Разделение сложной логики на понятные шаги
WITH 
raw_data AS (
    SELECT * FROM source_table 
    WHERE quality_check() AND date_filter()
),
cleaned_data AS (
    SELECT 
        id,
        COALESCE(value, 0) AS value,
        UPPER(TRIM(name)) AS name
    FROM raw_data
),
aggregated_data AS (
    SELECT 
        category,
        COUNT(*) AS count,
        AVG(value) AS avg_value,
        SUM(value) AS total_value
    FROM cleaned_data
    GROUP BY category
)
SELECT * FROM aggregated_data 
WHERE count > 10
ORDER BY total_value DESC;
```

#### ✅ **Тестировать части запроса через CTE**

```sql
-- Отладка: можно выполнять части запроса по отдельности
WITH 
-- Тестируем этот блок отдельно
test_block AS (
    SELECT 
        user_id,
        COUNT(*) AS login_count
    FROM login_logs
    WHERE login_date >= CURRENT_DATE - INTERVAL '7 days'
    GROUP BY user_id
    -- Можно выполнить: SELECT * FROM test_block
)
SELECT 
    u.name,
    tb.login_count,
    CASE 
        WHEN tb.login_count > 10 THEN 'Ежедневно'
        WHEN tb.login_count > 3 THEN 'Часто'
        ELSE 'Редко'
    END AS frequency
FROM test_block tb
JOIN users u ON tb.user_id = u.id;
```

#### ✅ **Использовать рекурсивные CTE для иерархий**

```sql
-- Стандартный шаблон для иерархических данных
WITH RECURSIVE hierarchy AS (
    -- Anchor: корневые элементы
    SELECT 
        id,
        parent_id,
        name,
        1 AS level,
        ARRAY[id] AS path_ids,
        name::TEXT AS path_names
    FROM categories
    WHERE parent_id IS NULL
    
    UNION ALL
    
    -- Recursive: дочерние элементы
    SELECT 
        c.id,
        c.parent_id,
        c.name,
        h.level + 1,
        h.path_ids || c.id,
        h.path_names || ' > ' || c.name
    FROM categories c
    JOIN hierarchy h ON c.parent_id = h.id
)
SELECT 
    id,
    LPAD('', (level - 1) * 2, ' ') || name AS indented_name,
    level,
    path_names
FROM hierarchy
ORDER BY path_ids;
```

#### ✅ **Ограничивать рекурсивные CTE для безопасности**

```sql
-- Всегда добавляйте защиту от бесконечной рекурсии
WITH RECURSIVE safe_recursion AS (
    SELECT ... WHERE ...  -- Anchor
    
    UNION ALL
    
    SELECT ... FROM safe_recursion
    WHERE 
        condition  -- Основное условие продолжения
        AND depth < 100  -- Максимальная глубина
        AND NOT cycle_detected  -- Проверка циклов
        AND array_length(path, 1) < 50  -- Максимальная длина пути
)
SELECT ... FROM safe_recursion;
```

### **Различия MySQL и PostgreSQL**

| Особенность | MySQL (8.0+) | PostgreSQL |
| :--- | :--- | :--- |
| **Синтаксис CTE** | `WITH cte AS (...)` | `WITH [RECURSIVE] cte AS (...)` |
| **Рекурсивные CTE** | ✅ Поддерживается | ✅ Полная поддержка |
| **Материализация** | Автоматическая, можно влиять через хинты | `MATERIALIZED` / `NOT MATERIALIZED` |
| **Ограничение рекурсии** | `cte_max_recursion_depth` (по умолчанию 1000) | MAXRECURSION нет, но есть глубины рекурсии |
| **Изменение данных в CTE** | ❌ Нельзя в рекурсивной части CTE | ✅ Можно (`WITH ... UPDATE/DELETE`) |
| **Несколько CTE** | ✅ Да, через запятую | ✅ Да, через запятую |
| **Оконные функции в CTE** | ✅ Поддерживаются | ✅ Поддерживаются |

#### **Пример различий:**

```sql
-- MySQL: лимит рекурсии
SET SESSION cte_max_recursion_depth = 10000;

WITH RECURSIVE numbers AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM numbers WHERE n < 10000
)
SELECT * FROM numbers;

-- PostgreSQL: материализация
WITH RECURSIVE hierarchy AS (
    SELECT id, parent_id FROM items WHERE parent_id IS NULL
    UNION ALL
    SELECT i.id, i.parent_id 
    FROM items i
    JOIN hierarchy h ON i.parent_id = h.id
)
SELECT * FROM hierarchy;
```

### **Шпаргалка раздела 6**

#### **Базовый синтаксис:**

```sql
WITH [RECURSIVE] имя_cte [(столбец1, столбец2, ...)] AS (
    SELECT ...  -- Определение CTE
)
SELECT ... FROM имя_cte;  -- Использование CTE
```

#### **Рекурсивный CTE:**

```sql
WITH RECURSIVE cte AS (
    -- Базовый запрос (anchor)
    SELECT ... FROM table WHERE начальное_условие
    
    UNION [ALL]
    
    -- Рекурсивный запрос
    SELECT ... FROM cte JOIN table ON условие_рекурсии
    WHERE условие_останова
)
SELECT * FROM cte;
```

#### **Когда использовать CTE:**
*   Улучшение читаемости сложных запросов
*   Рекурсивные запросы (иерархии, графы, последовательности)
*   Несколько обращений к одному результату в запросе
*   Пошаговая обработка данных (ETL в одном запросе)
*   Отладка сложных запросов

#### **Когда НЕ использовать CTE:**
*   Очень простые запросы
*   Когда нужна максимальная производительность (иногда подзапросы быстрее)
*   Когда результат нужен за пределами одного запроса (используйте временные таблицы)
*   Для очень больших объемов данных

#### **Оптимизация CTE:**
*   Для больших CTE используйте `MATERIALIZED` (PostgreSQL)
*   Для рекурсивных CTE добавляйте условия остановки
*   Используйте индексы в таблицах внутри CTE
*   Избегайте нескольких ссылок на большие CTE
*   Разбивайте очень сложные CTE на временные таблицы

#### **Примеры задач:**

```sql
-- 1. Найти всех подчиненных заданного менеджера
WITH RECURSIVE subordinates AS (
    SELECT id, name, manager_id FROM employees WHERE id = 123  -- ID менеджера
    UNION ALL
    SELECT e.id, e.name, e.manager_id
    FROM employees e
    JOIN subordinates s ON e.manager_id = s.id
)
SELECT * FROM subordinates WHERE id != 123;

-- 2. Генерация отчета по месяцам
WITH RECURSIVE months AS (
    SELECT DATE('2024-01-01') AS month_start
    UNION ALL
    SELECT DATE_ADD(month_start, INTERVAL 1 MONTH)
    FROM months
    WHERE month_start < '2024-12-01'
)
SELECT 
    m.month_start,
    COALESCE(SUM(o.amount), 0) AS monthly_sales
FROM months m
LEFT JOIN orders o ON 
    o.order_date >= m.month_start 
    AND o.order_date < DATE_ADD(m.month_start, INTERVAL 1 MONTH)
GROUP BY m.month_start
ORDER BY m.month_start;

-- 3. Поиск циклических зависимостей
WITH RECURSIVE dependency_chain AS (
    SELECT 
        from_table, 
        to_table, 
        ARRAY[from_table, to_table] AS path,
        from_table = to_table AS is_cycle
    FROM table_dependencies
    UNION ALL
    SELECT 
        dc.from_table,
        td.to_table,
        dc.path || td.to_table,
        td.to_table = ANY(dc.path)
    FROM dependency_chain dc
    JOIN table_dependencies td ON dc.to_table = td.from_table
    WHERE NOT dc.is_cycle
)
SELECT * FROM dependency_chain WHERE is_cycle;

-- 4. Развертка JSON массива (PostgreSQL)
WITH items AS (
    SELECT id, json_array_elements(data->'items') AS item
    FROM orders
)
SELECT 
    id,
    item->>'name' AS item_name,
    (item->>'price')::DECIMAL AS price
FROM items
WHERE (item->>'price')::DECIMAL > 100;
```

#### **Ключевые правила:**
*   CTE существуют только во время выполнения запроса
*   Рекурсивные CTE должны иметь условие остановки
*   CTE могут ссылаться на предыдущие CTE в том же `WITH`
*   В PostgreSQL можно управлять материализацией CTE
*   CTE не заменяют индексы — оптимизируйте запросы внутри CTE
*   Для очень больших данных рассмотрите временные таблицы
*   Используйте CTE для улучшения читаемости, а не для всех запросов подряд

## РАЗДЕЛ 7: ОКОННЫЕ ФУНКЦИИ

### **7.1. Основы оконных функций (OVER, PARTITION BY)**

**Что такое оконные функции?**
Оконные функции выполняют вычисления над набором строк (окном), но в отличие от агрегатных функций с GROUP BY, **не сворачивают строки в одну** — каждая строка сохраняет свою индивидуальность с добавленным вычисленным значением.

**Ключевые отличия от GROUP BY:**

| | GROUP BY | Оконные функции |
| :--- | :--- | :--- |
| **Сворачивание строк** | Сворачивает строки | Сохраняет все строки |
| **Результат** | Одна строка на группу | Столько же строк, сколько в исходных данных |
| **Доступ к значениям** | Невозможно обратиться к значениям других строк | Можно обращаться к соседним строкам |
| **Фильтрация** | `HAVING` для фильтрации групп | `WHERE` для фильтрации строк, затем оконная функция |

**Базовый синтаксис:**
```sql
Функция() OVER (
    [PARTITION BY столбец1, столбец2, ...]
    [ORDER BY столбец1 [ASC|DESC], ...]
    [ROWS/RANGE BETWEEN ...]
) AS имя_столбца
```

**OVER() — определение окна:**
```sql
-- Простейший пример: средняя зарплата для каждой строки
SELECT 
    name,
    salary,
    AVG(salary) OVER() AS company_avg_salary
FROM employees;

-- Результат:
-- | name     | salary | company_avg_salary |
-- |----------|--------|--------------------|
-- | Иван     | 75000  | 68333.33           |
-- | Анна     | 65000  | 68333.33           |
-- | Петр     | 82000  | 68333.33           |
-- | Мария    | 59000  | 68333.33           |
```

**PARTITION BY — разделение на группы:**
```sql
-- Средняя зарплата внутри каждого отдела
SELECT 
    name,
    department,
    salary,
    AVG(salary) OVER(PARTITION BY department) AS dept_avg_salary,
    salary - AVG(salary) OVER(PARTITION BY department) AS diff_from_dept_avg
FROM employees
ORDER BY department, salary DESC;

-- Несколько столбцов в PARTITION BY
SELECT 
    order_date,
    product_category,
    customer_region,
    amount,
    SUM(amount) OVER(PARTITION BY product_category, customer_region) AS region_category_total
FROM sales;
```

**ORDER BY внутри окна — определение порядка:**
```sql
-- Накопительная сумма по дате
SELECT 
    sale_date,
    amount,
    SUM(amount) OVER(ORDER BY sale_date) AS running_total
FROM sales
ORDER BY sale_date;

-- Результат:
-- | sale_date  | amount | running_total |
-- |------------|--------|---------------|
-- | 2024-01-01 | 1000   | 1000          |
-- | 2024-01-02 | 1500   | 2500          |
-- | 2024-01-03 | 800    | 3300          |
-- | 2024-01-04 | 2000   | 5300          |

-- ORDER BY влияет на поведение оконной функции:
-- Без ORDER BY: функция применяется ко всему окну
-- С ORDER BY: функция применяется нарастающим итогом (для некоторых функций)
```


### **7.2. Ранжирование (ROW_NUMBER, RANK, DENSE_RANK)**

**ROW_NUMBER() — последовательная нумерация:**
```sql
-- Нумерация сотрудников по зарплате в каждом отделе
SELECT 
    name,
    department,
    salary,
    ROW_NUMBER() OVER(PARTITION BY department ORDER BY salary DESC) AS salary_rank_in_dept
FROM employees;

-- Практическое использование: выбор топ-N записей
WITH ranked_products AS (
    SELECT 
        product_name,
        category,
        price,
        ROW_NUMBER() OVER(PARTITION BY category ORDER BY price DESC) AS rank_in_category
    FROM products
)
SELECT * FROM ranked_products WHERE rank_in_category <= 3;
```

**RANK() и DENSE_RANK() — ранжирование с учетом одинаковых значений:**
```sql
-- Пример данных с одинаковыми зарплатами
SELECT 
    name,
    salary,
    ROW_NUMBER() OVER(ORDER BY salary DESC) AS row_num,
    RANK() OVER(ORDER BY salary DESC) AS rank_with_gaps,
    DENSE_RANK() OVER(ORDER BY salary DESC) AS dense_rank_no_gaps
FROM employees;

-- Результат:
-- | name | salary | row_num | rank_with_gaps | dense_rank_no_gaps |
-- |------|--------|---------|----------------|--------------------|
-- | Иван | 100000 | 1       | 1              | 1                  |
-- | Анна | 90000  | 2       | 2              | 2                  |
-- | Петр | 90000  | 3       | 2              | 2                  |  ← Одинаковая зарплата
-- | Мария| 85000  | 4       | 4              | 3                  |  ← RANK пропустил 3, DENSE_RANK нет
-- | Олег | 80000  | 5       | 5              | 4                  |
```

**Сравнение функций ранжирования:**
*   **ROW_NUMBER():** Уникальный номер для каждой строки, даже при одинаковых значениях
*   **RANK():** Пропускает номера при одинаковых значениях (1, 2, 2, 4)
*   **DENSE_RANK():** Не пропускает номера при одинаковых значениях (1, 2, 2, 3)

**NTILE() — разбиение на группы:**
```sql
-- Разделить сотрудников на 4 группы по зарплате (квартили)
SELECT 
    name,
    salary,
    NTILE(4) OVER(ORDER BY salary DESC) AS salary_quartile,
    CASE NTILE(4) OVER(ORDER BY salary DESC)
        WHEN 1 THEN 'Высокий'
        WHEN 2 THEN 'Выше среднего'
        WHEN 3 THEN 'Ниже среднего'
        WHEN 4 THEN 'Низкий'
    END AS salary_level
FROM employees;

-- Разбиение внутри отделов
SELECT 
    name,
    department,
    salary,
    NTILE(3) OVER(PARTITION BY department ORDER BY salary DESC) AS tercile_in_dept
FROM employees;
```

### **7.3. Смещение (LAG, LEAD, FIRST_VALUE, LAST_VALUE)**

**LAG() и LEAD() — доступ к предыдущим и следующим строкам:**
```sql
-- Анализ последовательности продаж
SELECT 
    sale_date,
    amount,
    LAG(amount) OVER(ORDER BY sale_date) AS previous_day_sales,
    LEAD(amount) OVER(ORDER BY sale_date) AS next_day_sales,
    amount - LAG(amount) OVER(ORDER BY sale_date) AS day_over_day_change,
    ROUND(
        (amount * 100.0 / LAG(amount) OVER(ORDER BY sale_date)) - 100, 
        2
    ) AS day_over_day_percent
FROM sales
ORDER BY sale_date;

-- Результат:
-- | sale_date  | amount | prev | next | change | percent |
-- |------------|--------|------|------|--------|---------|
-- | 2024-01-01 | 1000   | NULL | 1500 | NULL   | NULL    |
-- | 2024-01-02 | 1500   | 1000 | 800  | 500    | 50.00   |
-- | 2024-01-03 | 800    | 1500 | 2000 | -700   | -46.67  |

-- LAG/LEAD с параметрами (смещение и значение по умолчанию)
SELECT 
    sale_date,
    amount,
    LAG(amount, 1, 0) OVER(ORDER BY sale_date) AS prev_day,  -- смещение 1, default 0
    LAG(amount, 7) OVER(ORDER BY sale_date) AS week_ago,      -- 7 дней назад
    LEAD(amount, 1, amount) OVER(ORDER BY sale_date) AS next_day  -- если NULL, берем текущее
FROM sales;
```

**FIRST_VALUE() и LAST_VALUE() — граничные значения:**
```sql
-- Сравнение с первой и последней продажей в месяце
SELECT 
    sale_date,
    amount,
    FIRST_VALUE(amount) OVER(PARTITION BY DATE_TRUNC('month', sale_date) ORDER BY sale_date) AS first_in_month,
    LAST_VALUE(amount) OVER(
        PARTITION BY DATE_TRUNC('month', sale_date) 
        ORDER BY sale_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS last_in_month,
    amount - FIRST_VALUE(amount) OVER(PARTITION BY DATE_TRUNC('month', sale_date) ORDER BY sale_date) AS diff_from_first
FROM sales
ORDER BY sale_date;

-- Важно: LAST_VALUE без указания фрейма вернет текущую строку, а не последнюю в окне!
-- Правильно: ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
```

**NTH_VALUE() — значение по порядковому номеру:**
```sql
-- Вторая по величине зарплата в каждом отделе
SELECT 
    name,
    department,
    salary,
    NTH_VALUE(salary, 2) OVER(
        PARTITION BY department 
        ORDER BY salary DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS second_highest_in_dept
FROM employees;
```

### **7.4. Агрегаты в окнах**

**Агрегатные функции с OVER():**
```sql
-- Комплексный анализ продаж
SELECT 
    sale_date,
    product_category,
    amount,
    -- Суммы
    SUM(amount) OVER(PARTITION BY product_category) AS category_total,
    SUM(amount) OVER() AS grand_total,
    
    -- Средние
    AVG(amount) OVER(PARTITION BY product_category) AS category_avg,
    AVG(amount) OVER() AS grand_avg,
    
    -- Доли
    ROUND(amount * 100.0 / SUM(amount) OVER(PARTITION BY product_category), 2) AS pct_of_category,
    ROUND(amount * 100.0 / SUM(amount) OVER(), 2) AS pct_of_total,
    
    -- Минимумы и максимумы
    MIN(amount) OVER(PARTITION BY product_category) AS category_min,
    MAX(amount) OVER(PARTITION BY product_category) AS category_max,
    
    -- Количество
    COUNT(*) OVER(PARTITION BY product_category) AS category_count
FROM sales
ORDER BY product_category, sale_date;
```

**Накопительные агрегаты (running totals):**
```sql
-- Накопительная сумма по разным окнам
SELECT 
    sale_date,
    customer_id,
    amount,
    -- Накопительная сумма по клиенту
    SUM(amount) OVER(
        PARTITION BY customer_id 
        ORDER BY sale_date
    ) AS customer_running_total,
    
    -- Накопительная сумма по всем данным
    SUM(amount) OVER(
        ORDER BY sale_date
    ) AS overall_running_total,
    
    -- Накопительная сумма за последние 3 продажи клиента
    SUM(amount) OVER(
        PARTITION BY customer_id 
        ORDER BY sale_date
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS last_3_sales_sum
FROM sales
ORDER BY customer_id, sale_date;
```

**Оконные функции с DISTINCT (в некоторых СУБД):**
```sql
-- PostgreSQL поддерживает COUNT(DISTINCT) в оконных функциях
SELECT 
    date,
    user_id,
    COUNT(DISTINCT user_id) OVER(ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS weekly_active_users
FROM logins;

-- В MySQL 8.0+ это не поддерживается, нужен обходной путь
WITH daily_users AS (
    SELECT DISTINCT date, user_id FROM logins
)
SELECT 
    date,
    COUNT(*) OVER(ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS weekly_active_users
FROM daily_users;
```

### **7.5. Фреймы (ROWS vs RANGE)**

**Что такое фрейм?**
Фрейм определяет, какие строки внутри окна участвуют в вычислении для текущей строки.

**Синтаксис фрейма:**
```sql
ROWS BETWEEN начало AND конец
-- или
RANGE BETWEEN начало AND конец
```

**ROWS vs RANGE:**

| Критерий | ROWS | RANGE |
| :--- | :--- | :--- |
| **Основа** | Физические строки | Логические значения |
| **Пример** | `ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING` | `RANGE BETWEEN 100 PRECEDING AND 100 FOLLOWING` |
| **Работа с одинаковыми значениями** | Каждая строка отдельно | Одинаковые значения обрабатываются вместе |
| **Производительность** | Обычно быстрее | Может быть медленнее |

**Примеры фреймов:**
```sql
-- Стандартные фреймы:
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW  -- от начала до текущей строки (по умолчанию с ORDER BY)
ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING  -- от текущей строки до конца
ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING  -- все строки в окне
ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING          -- 2 строки до и 2 после

-- Скользящее среднее за 3 дня
SELECT 
    date,
    temperature,
    AVG(temperature) OVER(
        ORDER BY date
        ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
    ) AS moving_avg_3days
FROM temperatures;

-- Разница ROWS vs RANGE
SELECT 
    score,
    COUNT(*) OVER(ORDER BY score ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS rows_count,
    COUNT(*) OVER(ORDER BY score RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS range_count
FROM test_scores;

-- Для одинаковых score:
-- ROWS: каждая строка считается отдельно
-- RANGE: строки с одинаковым score считаются как одна группа в текущем кадре
```

**Эффективное использование фреймов:**
```sql
-- Процент от накопительной суммы
SELECT 
    sale_date,
    amount,
    SUM(amount) OVER(ORDER BY sale_date) AS running_total,
    ROUND(
        amount * 100.0 / SUM(amount) OVER(ORDER BY sale_date),
        2
    ) AS pct_of_running_total,
    
    -- Процент от общей суммы
    ROUND(
        amount * 100.0 / SUM(amount) OVER(ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING),
        2
    ) AS pct_of_total
FROM sales;

-- Сравнение с предыдущим и следующим значением
SELECT 
    date,
    value,
    LAG(value) OVER(ORDER BY date) AS prev_value,
    value - LAG(value) OVER(ORDER BY date) AS diff_prev,
    LEAD(value) OVER(ORDER BY date) AS next_value,
    LEAD(value) OVER(ORDER BY date) - value AS diff_next
FROM metrics;
```


### **7.6. Практические паттерны**

**Топ-N запросов:**
```sql
-- 3 самых дорогих товара в каждой категории
SELECT * FROM (
    SELECT 
        product_name,
        category,
        price,
        ROW_NUMBER() OVER(PARTITION BY category ORDER BY price DESC) AS rank_in_category
    FROM products
) ranked
WHERE rank_in_category <= 3;

-- Альтернатива с использованием DENSE_RANK для учета одинаковых цен
SELECT * FROM (
    SELECT 
        product_name,
        category,
        price,
        DENSE_RANK() OVER(PARTITION BY category ORDER BY price DESC) AS rank
    FROM products
) ranked
WHERE rank <= 3;
```

**Дедупликация данных:**
```sql
-- Удаление дубликатов, оставляя самую новую запись
WITH numbered_records AS (
    SELECT 
        *,
        ROW_NUMBER() OVER(
            PARTITION BY user_id, email 
            ORDER BY created_at DESC
        ) AS rn
    FROM users
)
DELETE FROM users 
WHERE id IN (
    SELECT id FROM numbered_records WHERE rn > 1
);

-- Или создание таблицы без дубликатов
CREATE TABLE users_dedup AS
SELECT DISTINCT ON (user_id, email) *
FROM users
ORDER BY user_id, email, created_at DESC;
```

**Поиск gaps and islands (пропусков и последовательностей):**
```sql
-- Найти непрерывные периоды активности пользователей
WITH activity_groups AS (
    SELECT 
        user_id,
        activity_date,
        DATE_SUB(activity_date, INTERVAL 
            ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY activity_date) DAY
        ) AS grp
    FROM user_activity
)
SELECT 
    user_id,
    MIN(activity_date) AS period_start,
    MAX(activity_date) AS period_end,
    COUNT(*) AS days_active,
    DATEDIFF(MAX(activity_date), MIN(activity_date)) + 1 AS period_length
FROM activity_groups
GROUP BY user_id, grp
ORDER BY user_id, period_start;
```

**Расчет скользящих метрик:**
```sql
-- Скользящее среднее за 7 дней с исключением текущего дня
SELECT 
    date,
    revenue,
    AVG(revenue) OVER(
        ORDER BY date
        ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING
    ) AS moving_avg_7days_excluding_today,
    
    -- С медианой за 30 дней
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY revenue) OVER(
        ORDER BY date
        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) AS median_30days
FROM daily_revenue;
```

**Сравнение со средним по группе:**
```sql
-- Найти аномалии: значения, отличающиеся более чем на 2 стандартных отклонения
WITH stats AS (
    SELECT 
        metric,
        AVG(value) OVER(PARTITION BY metric) AS avg_value,
        STDDEV(value) OVER(PARTITION BY metric) AS std_value
    FROM measurements
)
SELECT 
    m.*,
    s.avg_value,
    s.std_value,
    CASE 
        WHEN ABS(m.value - s.avg_value) > 2 * s.std_value THEN 'Аномалия'
        ELSE 'Норма'
    END AS status
FROM measurements m
JOIN stats s ON m.metric = s.metric
WHERE ABS(m.value - s.avg_value) > 2 * s.std_value;
```

### **Антипаттерны раздела 7**

#### ❌ **Использование оконных функций там, где достаточно GROUP BY**

```sql
-- ПЛОХО: Оконные функции для простой агрегации
SELECT 
    department,
    AVG(salary) OVER(PARTITION BY department) AS avg_salary
FROM employees;

-- ✅ ХОРОШО: GROUP BY проще и эффективнее
SELECT 
    department,
    AVG(salary) AS avg_salary
FROM employees
GROUP BY department;
```

#### ❌ **Неправильное использование LAST_VALUE()**

```sql
-- ПЛОХО: LAST_VALUE без указания фрейма вернет текущую строку
SELECT 
    date,
    value,
    LAST_VALUE(value) OVER(ORDER BY date) AS wrong_last_value
FROM timeseries;

-- ✅ ХОРОШО: Указать полный фрейм
SELECT 
    date,
    value,
    LAST_VALUE(value) OVER(
        ORDER BY date
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS correct_last_value
FROM timeseries;
```

#### ❌ **Слишком сложные оконные функции в одном запросе**

```sql
-- ПЛОХО: Множество оконных функций с разными окнами
SELECT 
    *,
    ROW_NUMBER() OVER(PARTITION BY a ORDER BY b) AS rn1,
    RANK() OVER(PARTITION BY c ORDER BY d) AS rk1,
    LAG(value) OVER(PARTITION BY e ORDER BY f) AS lag1,
    SUM(amount) OVER(PARTITION BY g ORDER BY h) AS sum1
FROM table;

-- ✅ ХОРОШО: Разделить на несколько CTE или использовать именованные окна
WITH 
step1 AS (
    SELECT *, ROW_NUMBER() OVER(PARTITION BY a ORDER BY b) AS rn1
    FROM table
),
step2 AS (
    SELECT *, RANK() OVER(PARTITION BY c ORDER BY d) AS rk1
    FROM step1
)
SELECT 
    *,
    LAG(value) OVER(PARTITION BY e ORDER BY f) AS lag1
FROM step2;
```

#### ❌ **Игнорирование производительности на больших данных**

```sql
-- ПЛОХО: Оконные функции без индексов на PARTITION BY/ORDER BY столбцах
SELECT 
    user_id,
    event_date,
    LAG(event_date) OVER(PARTITION BY user_id ORDER BY event_date) AS prev_event
FROM huge_events_table;  -- 100M+ строк без индексов

-- ✅ ХОРОШО: Создать индексы
CREATE INDEX idx_user_date ON huge_events_table(user_id, event_date);
-- или использовать материализованные представления для агрегированных данных
```


### **Лучшие практики раздела 7**

#### ✅ **Использовать именованные окна для повторяющихся определений**

```sql
-- Дублирование кода
SELECT 
    SUM(amount) OVER(PARTITION BY department ORDER BY sale_date) AS dept_running,
    AVG(amount) OVER(PARTITION BY department ORDER BY sale_date) AS dept_avg,
    COUNT(*) OVER(PARTITION BY department ORDER BY sale_date) AS dept_count
FROM sales;

-- ✅ Чище с именованным окном
SELECT 
    SUM(amount) OVER w AS dept_running,
    AVG(amount) OVER w AS dept_avg,
    COUNT(*) OVER w AS dept_count
FROM sales
WINDOW w AS (PARTITION BY department ORDER BY sale_date);

-- Несколько именованных окон
SELECT 
    SUM(amount) OVER w1 AS dept_total,
    SUM(amount) OVER w2 AS overall_total,
    RANK() OVER w3 AS dept_rank
FROM sales
WINDOW 
    w1 AS (PARTITION BY department),
    w2 AS (),
    w3 AS (PARTITION BY department ORDER BY amount DESC);
```

#### ✅ **Правильно выбирать между ROWS и RANGE**

```sql
-- ROWS для физического смещения
SELECT 
    date,
    value,
    AVG(value) OVER(
        ORDER BY date
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS moving_avg  -- последние 3 строки

-- RANGE для логического диапазона
SELECT 
    date,
    value,
    AVG(value) OVER(
        ORDER BY date
        RANGE BETWEEN INTERVAL '2' DAY PRECEDING AND CURRENT ROW
    ) AS moving_avg  -- значения за последние 3 дня
FROM time_series;
```

#### ✅ **Создавать индексы для оконных функций**

```sql
-- Индексы для ускорения оконных функций
CREATE INDEX idx_window_perf ON sales(department, sale_date, amount);

-- Составные индексы в порядке PARTITION BY затем ORDER BY
CREATE INDEX idx_dept_date ON employees(department, hire_date, salary);
-- Для: ROW_NUMBER() OVER(PARTITION BY department ORDER BY hire_date)
```

#### ✅ **Использовать EXPLAIN для анализа производительности**

```sql
-- Анализ плана выполнения оконных функций
EXPLAIN 
SELECT 
    department,
    salary,
    AVG(salary) OVER(PARTITION BY department) AS dept_avg
FROM employees;

-- В выводе смотрите:
-- 1. WindowAgg операция
-- 2. Sort операции (дорогие)
-- 3. Используются ли индексы
-- 4. Размер Window буфера
```

### **Различия MySQL и PostgreSQL**

| Функциональность | MySQL (8.0+) | PostgreSQL |
| :--- | :--- | :--- |
| **Поддержка оконных функций** | Полная с 8.0 | Полная |
| **Именованные окна (WINDOW clause)** | ✅ Да | ✅ Да |
| **RANGE с datetime интервалами** | ❌ Нет | ✅ Да |
| **GROUPS фрейм** | ✅ 8.0+ | ✅ Да |
| **EXCLUDE в фреймах** | ❌ Нет | ✅ Да |
| **FILTER в агрегатных оконных функциях** | ❌ Нет | ✅ Да |
| **Производительность** | Хорошая | Очень хорошая |

#### **Пример FILTER в PostgreSQL:**

```sql
-- Условные агрегаты в окнах
SELECT 
    date,
    category,
    amount,
    SUM(amount) OVER(PARTITION BY category) AS total_category,
    SUM(amount) FILTER (WHERE amount > 100) OVER(PARTITION BY category) AS total_large_transactions
FROM sales;
```

### **Шпаргалка раздела 7**

#### **Базовый синтаксис:**

```sql
функция() OVER (
    [PARTITION BY столбцы]
    [ORDER BY столбцы [ASC|DESC]]
    [фрейм]
) AS имя_столбца
```

#### **Основные оконные функции:**

| Категория | Функции | Назначение |
| :--- | :--- | :--- |
| **Ранжирование** | `ROW_NUMBER()`, `RANK()`, `DENSE_RANK()`, `NTILE(n)` | Нумерация и ранжирование строк |
| **Смещение** | `LAG()`, `LEAD()`, `FIRST_VALUE()`, `LAST_VALUE()`, `NTH_VALUE()` | Доступ к соседним строкам |
| **Агрегаты** | `SUM()`, `AVG()`, `COUNT()`, `MIN()`, `MAX()` | Агрегация по окну |
| **Статистические** | `STDDEV()`, `VARIANCE()`, `PERCENTILE_CONT()` | Статистические расчеты |

#### **Фреймы (ROWS):**
*   `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW` — от начала до текущей
*   `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING` — от текущей до конца
*   `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` — все строки
*   `ROWS BETWEEN n PRECEDING AND m FOLLOWING` — диапазон вокруг текущей

#### **Порядок выполнения:**
1.  `FROM`, `JOIN`, `WHERE`
2.  `GROUP BY`, агрегаты
3.  Оконные функции
4.  `SELECT`
5.  `DISTINCT`
6.  `ORDER BY`
7.  `LIMIT`

#### **Типичные задачи и решения:**

```sql
-- 1. Топ-N в каждой группе
SELECT * FROM (
    SELECT *, ROW_NUMBER() OVER(PARTITION BY group_col ORDER BY value DESC) AS rn
    FROM table
) t WHERE rn <= N;

-- 2. Разница с предыдущим значением
SELECT 
    value - LAG(value) OVER(ORDER BY date) AS diff
FROM table;

-- 3. Накопительная сумма
SELECT 
    SUM(amount) OVER(ORDER BY date) AS running_total
FROM sales;

-- 4. Дедупликация
DELETE FROM table 
WHERE id IN (
    SELECT id FROM (
        SELECT id, ROW_NUMBER() OVER(PARTITION BY dup_cols ORDER BY created_at DESC) AS rn
        FROM table
    ) t WHERE rn > 1
);

-- 5. Скользящее среднее
SELECT 
    AVG(value) OVER(ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS weekly_avg
FROM metrics;

-- 6. Доля в группе
SELECT 
    amount * 100.0 / SUM(amount) OVER(PARTITION BY group_col) AS pct_of_group
FROM table;

-- 7. Пропуски в последовательностях (gaps)
WITH numbered AS (
    SELECT 
        date,
        ROW_NUMBER() OVER(ORDER BY date) AS rn
    FROM dates
)
SELECT 
    DATE_ADD(MIN(date), INTERVAL rn DAY) AS missing_date
FROM numbered
GROUP BY rn
HAVING MIN(date) != DATE_ADD('2024-01-01', INTERVAL rn DAY);
```

#### **Оптимизация оконных функций:**
*   Используйте индексы на столбцах в `PARTITION BY` и `ORDER BY`
*   Избегайте оконных функций на больших таблицах без фильтрации
*   Для сложных вычислений используйте материализованные представления
*   Проверяйте план выполнения с `EXPLAIN`
*   Используйте `LIMIT` для уменьшения объема обрабатываемых данных


#### **Ключевые правила:**
*   Оконные функции выполняются после `WHERE`, `GROUP BY`, `HAVING`
*   `PARTITION BY` делит данные на независимые окна
*   `ORDER BY` внутри `OVER` определяет порядок обработки строк
*   Фрейм определяет, какие строки участвуют в вычислении для текущей строки
*   Оконные функции не сворачивают строки — результат имеет столько же строк, сколько исходные данные
*   Индексы критически важны для производительности оконных функций

## РАЗДЕЛ 8: ОПТИМИЗАЦИЯ И ИНДЕКСЫ

### **8.1. Типы индексов (B-дерево, хэш, полнотекстовый)**

**Что такое индекс?**
Индекс — это отдельная структура данных, которая ускоряет поиск данных в таблице за счет дополнительных расходов на запись и хранение. Принцип аналогии: как оглавление в книге помогает быстро найти нужную страницу.

**B-дерево индекс (наиболее распространенный)**
```sql
-- Создание B-дерево индекса
CREATE INDEX idx_users_email ON users(email);
CREATE INDEX idx_orders_date_customer ON orders(order_date, customer_id);

-- Пример структуры B-дерево:
--       [Рут: 50]
--       /       \
--   [<50]       [≥50]
--   /   \       /   \
-- [20,30,40] [60,70,80]
```
**Преимущества:**
*   Поддерживает диапазонные запросы (`BETWEEN`, `>`, `<`)
*   Поддерживает сортировку (`ORDER BY`)
*   Эффективен для высокоселективных столбцов

**Хэш-индекс**
```sql
-- Только для точного сравнения (=)
CREATE INDEX idx_users_id_hash ON users USING HASH(id);
```
**Особенности:**
*   Только для операций `=` (не для `>`, `<`, `BETWEEN`)
*   Быстрее B-дерево для точных совпадений
*   Не поддерживает сортировку
*   Не рекомендуется для часто изменяемых данных

**Полнотекстовый индекс (для поиска по тексту)**
```sql
-- MySQL
CREATE FULLTEXT INDEX idx_products_description ON products(description);

-- PostgreSQL
CREATE INDEX idx_products_fts ON products USING GIN(to_tsvector('russian', description));

-- Поиск
SELECT * FROM products 
WHERE MATCH(description) AGAINST('ноутбук быстрый' IN BOOLEAN MODE);  -- MySQL

SELECT * FROM products 
WHERE to_tsvector('russian', description) @@ to_tsquery('russian', 'ноутбук & быстрый');  -- PG
```

**Другие типы индексов:**
*   **GiST/SP-GiST** (PostgreSQL): Для геоданных, полнотекстового поиска
*   **BRIN** (PostgreSQL): Для больших таблиц с естественной сортировкой (логи, временные ряды)
*   **R-дерево**: Для пространственных данных

### **8.2. Составные и покрывающие индексы**

**Составной индекс (индекс по нескольким столбцам)**
```sql
-- Создание составного индекса
CREATE INDEX idx_orders_date_status_customer ON orders(order_date, status, customer_id);

-- Когда используется эффективно:
SELECT * FROM orders WHERE order_date = '2024-01-15' AND status = 'completed';
SELECT * FROM orders WHERE order_date = '2024-01-15' AND status = 'completed' AND customer_id = 123;
SELECT * FROM orders WHERE order_date = '2024-01-15' ORDER BY status;

-- Когда используется НЕ эффективно (пропуск первого столбца):
SELECT * FROM orders WHERE status = 'completed';  -- Индекс не используется
SELECT * FROM orders WHERE customer_id = 123;      -- Индекс не используется
```

**Правило левого префикса:**
```
Индекс (A, B, C) эффективен для запросов с:
- A
- A, B
- A, B, C

Но НЕ эффективен для:
- B
- C
- B, C
```

**Покрывающий индекс (Covering Index)**
```sql
-- Содержит ВСЕ поля, необходимые для запроса
-- MySQL: все поля в ключе
CREATE INDEX idx_covering ON orders(order_date, customer_id, amount, status);

-- PostgreSQL: используем INCLUDE для непокрывающих полей
CREATE INDEX idx_covering ON orders(order_date, customer_id) INCLUDE (amount, status);

-- Запрос использует только индекс (Index Only Scan)
SELECT order_date, customer_id, amount 
FROM orders 
WHERE order_date BETWEEN '2024-01-01' AND '2024-01-31';
```

**Плюсы покрывающего индекса:**
*   Не нужно читать саму таблицу (только индекс)
*   Меньше операций ввода/вывода
*   Быстрее на 2-10 раз

**Минусы:**
*   Больше места на диске
*   Медленнее вставки/обновления


### **8.3. Кластеризованные vs некластеризованные индексы**

**Кластеризованный индекс**
```sql
-- В MySQL (InnoDB) PRIMARY KEY всегда кластеризованный
CREATE TABLE users (
    id INT PRIMARY KEY,  -- Кластеризованный индекс
    email VARCHAR(100)
);

-- В SQL Server можно указать явно
CREATE TABLE orders (
    order_id INT,
    order_date DATE,
    CLUSTERED INDEX idx_clustered (order_date)  -- Данные физически упорядочены по дате
);
```
**Характеристики:**
*   Только один на таблицу
*   Определяет физический порядок данных на диске
*   Быстрее для диапазонных запросов
*   PRIMARY KEY обычно кластеризованный

**Некластеризованный индекс**
```sql
-- Все индексы кроме PRIMARY KEY
CREATE INDEX idx_users_email ON users(email);  -- Некластеризованный
CREATE UNIQUE INDEX idx_users_phone ON users(phone);  -- Уникальный некластеризованный
```

**Сравнение:**

| Критерий | Кластеризованный | Некластеризованный |
| :--- | :--- | :--- |
| **Количество** | 1 на таблицу | Много на таблицу |
| **Скорость чтения** | Очень быстрая (данные рядом) | Быстрая (двойной доступ) |
| **Скорость записи** | Медленная (переупорядочивание) | Быстрая |
| **Доп. место** | Не занимает (это данные) | Занимает отдельное место |
| **Типичное использование** | PRIMARY KEY, часто в диапазонных запросах | Все остальные индексы |

### **8.4. План выполнения (EXPLAIN, EXPLAIN ANALYZE)**

**EXPLAIN — предварительный план**
```sql
-- MySQL
EXPLAIN SELECT * FROM users WHERE email = 'test@example.com';

-- PostgreSQL
EXPLAIN SELECT * FROM users WHERE email = 'test@example.com';

-- Вывод показывает:
-- id: порядок выполнения
-- select_type: тип операции
-- table: таблица
-- type: тип доступа (index, range, ref, ALL)
-- possible_keys: возможные индексы
-- key: выбранный индекс
-- rows: оценка строк
-- Extra: дополнительная информация
```

**EXPLAIN ANALYZE — реальное выполнение**
```sql
-- PostgreSQL (выполняет запрос и показывает фактические данные)
EXPLAIN ANALYZE SELECT * FROM large_table WHERE date > '2024-01-01';

-- MySQL 8.0+
EXPLAIN ANALYZE SELECT * FROM large_table WHERE date > '2024-01-01';

-- Вывод включает:
-- Фактическое время выполнения
-- Фактическое количество строк
-- Реальные буферы в памяти
```

**Чтение ключевых полей в EXPLAIN:**

**type** (тип доступа, от лучшего к худшему):
*   `system` / `const`: Одна строка
*   `eq_ref`: Уникальный поиск по индексу
*   `ref`: Поиск по не-уникальному индексу
*   `range`: Диапазон по индексу
*   `index`: Полное сканирование индекса
*   `ALL`: Полное сканирование таблицы

**Extra** (важные флаги):
*   `Using index`: Покрывающий индекс (только индекс)
*   `Using where`: Фильтрация после чтения
*   `Using temporary`: Временная таблица
*   `Using filesort`: Сортировка на диске (медленно)
*   `Using index condition`: Индексный предикат

**Пример анализа:**
```sql
EXPLAIN 
SELECT o.order_date, c.name, SUM(o.amount)
FROM orders o
JOIN customers c ON o.customer_id = c.id
WHERE o.status = 'completed'
  AND o.order_date >= '2024-01-01'
GROUP BY o.order_date, c.name
ORDER BY o.order_date;

-- Что смотреть:
-- 1. Порядок JOIN (начинается с самой маленькой таблицы?)
-- 2. Типы доступа (range, ref лучше чем ALL)
-- 3. Используются ли индексы
-- 4. Есть ли временные таблицы или сортировка на диске
```

### **8.5. Когда индекс не используется**

#### **1. Неселективные условия (слишком много строк)**
```sql
-- Плохо: индекс не используется если >15-20% строк
SELECT * FROM users WHERE gender = 'M';  -- 50% строк

-- Решение: не создавать индекс на неселективные столбцы
-- или использовать составные индексы
CREATE INDEX idx_users_gender_country ON users(gender, country);
SELECT * FROM users WHERE gender = 'M' AND country = 'RU';  -- Теперь селективно
```

#### **2. Функции или выражения над полями**
```sql
-- ❌ Индекс не используется
SELECT * FROM users WHERE UPPER(email) = 'TEST@EXAMPLE.COM';
SELECT * FROM orders WHERE YEAR(order_date) = 2024;
SELECT * FROM products WHERE price * 1.1 > 100;

-- ✅ Переписать без функций
SELECT * FROM users WHERE email = 'test@example.com';
SELECT * FROM orders WHERE order_date BETWEEN '2024-01-01' AND '2024-12-31';
SELECT * FROM products WHERE price > 100 / 1.1;

-- ✅ Или создать индекс-выражение (PostgreSQL)
CREATE INDEX idx_users_email_upper ON users(UPPER(email));
CREATE INDEX idx_orders_year ON orders(EXTRACT(YEAR FROM order_date));
```

#### **3. LIKE с начальным wildcard**
```sql
-- ❌ Индекс не используется
SELECT * FROM products WHERE name LIKE '%ноутбук%';

-- ✅ Использовать полнотекстовый поиск
CREATE FULLTEXT INDEX idx_products_name ON products(name);
SELECT * FROM products WHERE MATCH(name) AGAINST('ноутбук');

-- ✅ Или поиск по префиксу (индекс используется)
SELECT * FROM products WHERE name LIKE 'ноутбук%';
```

#### **4. Неявное преобразование типов**
```sql
-- ❌ Индекс не используется (если phone VARCHAR)
SELECT * FROM users WHERE phone = 1234567890;

-- ✅ Использовать правильный тип
SELECT * FROM users WHERE phone = '1234567890';

-- ❌ Даты как строки
SELECT * FROM orders WHERE order_date = '2024-01-15';  -- OK если order_date DATE
SELECT * FROM logs WHERE timestamp_col = '2024-01-15 10:30:00';  -- OK если совпадают типы
```

#### **5. OR вместо IN или UNION**
```sql
-- ❌ Может не использовать индекс оптимально
SELECT * FROM products 
WHERE category = 'Электроника' 
   OR category = 'Бытовая техника';

-- ✅ Использовать IN
SELECT * FROM products 
WHERE category IN ('Электроника', 'Бытовая техника');

-- ✅ Или UNION для разных индексов
SELECT * FROM products WHERE category = 'Электроника'
UNION ALL
SELECT * FROM products WHERE category = 'Бытовая техника';
```

### **8.6. Оптимизация JOIN и подзапросов**

**Оптимизация JOIN:**
```sql
-- 1. Порядок таблиц имеет значение
-- Начинать с самой маленькой таблицы
SELECT * FROM small_table s
JOIN large_table l ON s.id = l.small_id;  -- Лучше

SELECT * FROM large_table l
JOIN small_table s ON l.small_id = s.id;  -- Хуже

-- 2. Индексы на полях соединения
CREATE INDEX idx_orders_customer ON orders(customer_id);
CREATE INDEX idx_customers_id ON customers(id);  -- Обычно PK

-- 3. Фильтровать ДО соединения
-- Плохо: сначала JOIN, потом WHERE
SELECT * FROM orders o
JOIN customers c ON o.customer_id = c.id
WHERE o.amount > 1000 AND c.country = 'RU';

-- Лучше: фильтровать до JOIN
SELECT * FROM 
    (SELECT * FROM orders WHERE amount > 1000) o
JOIN 
    (SELECT * FROM customers WHERE country = 'RU') c 
ON o.customer_id = c.id;
```

**Оптимизация подзапросов:**
```sql
-- ❌ Коррелированный подзапрос (медленно)
SELECT name, 
       (SELECT COUNT(*) FROM orders o WHERE o.customer_id = c.id) AS order_count
FROM customers c;

-- ✅ JOIN с группировкой (быстрее)
SELECT c.name, COUNT(o.id) AS order_count
FROM customers c
LEFT JOIN orders o ON c.id = o.customer_id
GROUP BY c.id, c.name;

-- ❌ IN с большим подзапросом
SELECT * FROM products 
WHERE category_id IN (SELECT id FROM categories WHERE active = true);

-- ✅ EXISTS может быть быстрее
SELECT * FROM products p
WHERE EXISTS (SELECT 1 FROM categories c WHERE c.id = p.category_id AND c.active = true);

-- ✅ Или JOIN
SELECT p.* 
FROM products p
JOIN categories c ON p.category_id = c.id AND c.active = true;
```

**Оптимизация UNION:**
```sql
-- ❌ UNION удаляет дубликаты (дорого)
SELECT id FROM table1 WHERE condition1
UNION
SELECT id FROM table2 WHERE condition2;

-- ✅ UNION ALL если дубликаты не важны (быстрее)
SELECT id FROM table1 WHERE condition1
UNION ALL
SELECT id FROM table2 WHERE condition2;

-- ✅ Или агрегация после UNION ALL если нужны уникальные
SELECT DISTINCT id FROM (
    SELECT id FROM table1 WHERE condition1
    UNION ALL
    SELECT id FROM table2 WHERE condition2
) combined;
```

### **Антипаттерны раздела 8**

#### ❌ **Слишком много индексов**

```sql
-- ПЛОХО: Индекс на каждый столбец
CREATE INDEX idx_users_name ON users(name);
CREATE INDEX idx_users_email ON users(email);
CREATE INDEX idx_users_phone ON users(phone);
CREATE INDEX idx_users_birthdate ON users(birthdate);
CREATE INDEX idx_users_city ON users(city);
-- Каждый INSERT/UPDATE/DELETE должен обновлять все индексы!

-- ✅ ХОРОШО: Только необходимые индексы + составные
CREATE INDEX idx_users_email ON users(email);  -- Для поиска по email
CREATE INDEX idx_users_city_birthdate ON users(city, birthdate);  -- Для запросов по городу и дате
```

#### ❌ **Индексы на часто изменяемые данные**

```sql
-- ПЛОХО: Индекс на столбец, который меняется каждую минуту
CREATE INDEX idx_sensors_current_value ON sensors(current_value);
-- current_value обновляется каждые 10 секунд
-- Индекс постоянно перестраивается

-- ✅ ХОРОШО: Индексировать стабильные или низкочастотные изменения
CREATE INDEX idx_sensors_sensor_id ON sensors(sensor_id);
CREATE INDEX idx_sensors_timestamp ON sensors(timestamp);
```

#### ❌ **Составной индекс в неправильном порядке**

```sql
-- ПЛОХО: Порядок не соответствует запросам
CREATE INDEX idx_orders_customer_date ON orders(customer_id, order_date);
-- Но часто запрашивают: WHERE order_date > '2024-01-01'

-- ✅ ХОРОШО: Сначала более селективный и часто используемый
CREATE INDEX idx_orders_date_customer ON orders(order_date, customer_id);
-- Работает для:
-- WHERE order_date > '2024-01-01'
-- WHERE order_date = '2024-01-15' AND customer_id = 123
```

#### ❌ **Игнорирование статистики**

```sql
-- ПЛОХО: Старая статистика → плохой план выполнения
-- Таблица выросла с 1000 до 1,000,000 строк
-- Статистика не обновлена
-- Оптимизатор думает, что таблица маленькая и выбирает плохой план

-- ✅ ХОРОШО: Регулярно обновлять статистику
-- PostgreSQL
ANALYZE table_name;
VACUUM ANALYZE table_name;

-- MySQL (InnoDB)
ANALYZE TABLE table_name;
OPTIMIZE TABLE table_name;  -- Для MyISAM
```


### **Лучшие практики раздела 8**

#### ✅ **Создавать индексы на основе реальных запросов**

```sql
-- 1. Анализировать медленные запросы
SHOW FULL PROCESSLIST;  -- MySQL
SELECT * FROM pg_stat_activity WHERE state = 'active';  -- PostgreSQL

-- 2. Использовать slow query log
-- MySQL: slow_query_log = 1, long_query_time = 2
-- PostgreSQL: log_min_duration_statement = 2000

-- 3. Анализировать с помощью EXPLAIN
EXPLAIN ANALYZE 
SELECT * FROM orders 
WHERE customer_id = 123 
  AND status = 'completed' 
  AND order_date >= '2024-01-01';

-- 4. Создавать индекс под конкретный запрос
CREATE INDEX idx_orders_customer_status_date 
ON orders(customer_id, status, order_date);
```

#### ✅ **Мониторинг использования индексов**

```sql
-- MySQL: проверка использования индексов
SELECT 
    object_schema,
    object_name,
    index_name,
    count_read,
    count_fetch
FROM performance_schema.table_io_waits_summary_by_index_usage
WHERE index_name IS NOT NULL
ORDER BY count_read DESC;

-- PostgreSQL: какие индексы не используются
SELECT 
    schemaname,
    tablename,
    indexname,
    idx_scan  -- Количество сканирований
FROM pg_stat_user_indexes
WHERE idx_scan = 0;  -- Никогда не использовались

-- Удалить неиспользуемые индексы
DROP INDEX idx_unused_index;
```

#### ✅ **Использовать частичные индексы (PostgreSQL)**

```sql
-- Индекс только для части данных
CREATE INDEX idx_orders_active ON orders(status) WHERE status = 'active';
-- Занимает меньше места, быстрее

-- Индекс для часто запрашиваемого подмножества
CREATE INDEX idx_recent_logs ON logs(timestamp) 
WHERE timestamp > CURRENT_DATE - INTERVAL '30 days';

-- Уникальность для части данных
CREATE UNIQUE INDEX idx_users_active_email ON users(email) 
WHERE active = true;
```

#### ✅ **Оптимизировать размер индекса**

```sql
-- Индекс на часть строки (prefix index)
-- MySQL
CREATE INDEX idx_products_name_prefix ON products(name(50));  -- Первые 50 символов

-- Индекс на выражение (меньше места)
CREATE INDEX idx_users_lower_email ON users(LOWER(email));

-- Выбирать оптимальный тип данных для индекса
-- Плохо: VARCHAR(255) для индекса, если средняя длина 10 символов
-- Лучше: VARCHAR(50) или даже CHAR(10) если длина фиксирована

-- Сжатие индексов (зависит от СУБД)
CREATE INDEX idx_large_table ON large_table(column1, column2) 
WITH (compression = low);  -- PostgreSQL
```

### **Различия MySQL и PostgreSQL**

| Оптимизация | MySQL (InnoDB) | PostgreSQL |
| :--- | :--- | :--- |
| **Кластеризованный индекс** | Только PRIMARY KEY | Можно выбрать любой индекс с CLUSTER |
| **Покрывающие индексы** | Все поля в ключе | INCLUDE для дополнительных полей |
| **Частичные индексы** | Не поддерживается | ✅ Полная поддержка |
| **Индексные выражения** | С версии 8.0 | ✅ Полная поддержка |
| **Параллельные запросы** | Ограниченно | ✅ Хорошая поддержка |
| **Статистика** | `ANALYZE TABLE` | `ANALYZE`, авто-вакуум |
| **Оптимизатор** | Cost-based, иногда простой | Очень сложный, cost-based |
| **Джойны** | Nested Loop, Hash Join (8.0+) | Nested Loop, Hash, Merge Join |

#### **Примеры специфичных оптимизаций:**

```sql
-- MySQL: индекс на геоданные
CREATE SPATIAL INDEX idx_locations_coords ON locations(coordinates);

-- PostgreSQL: BRIN индекс для временных рядов
CREATE INDEX idx_logs_time_brin ON logs USING BRIN(log_time);

-- PostgreSQL: GiST индекс для полнотекстового поиска
CREATE INDEX idx_docs_content ON docs USING GIN(to_tsvector('russian', content));
```

### **Шпаргалка раздела 8**

#### **Когда создавать индекс:**
*   Столбцы в WHERE с высокой селективностью (> 5% уникальных значений)
*   Столбцы в JOIN условиях
*   Столбцы в ORDER BY или GROUP BY
*   Первичные и внешние ключи (автоматически в большинстве СУБД)

#### **Когда НЕ создавать индекс:**
*   Маленькие таблицы (< 1000 строк)
*   Часто изменяемые столбцы
*   Неселективные столбцы (пол, статус с 2-3 значениями)
*   Когда уже есть похожий составной индекс

#### **Типы индексов по применению:**
*   **B-дерево:** Универсальный, для сравнений, диапазонов, сортировки
*   **Хэш:** Только точное совпадение (=), быстрый
*   **Полнотекстовый:** Поиск по тексту
*   **Составной:** Для условий по нескольким столбцам
*   **Покрывающий:** Содержит все поля запроса

#### **Анализ производительности:**

```sql
-- 1. Найти медленные запросы
-- MySQL
SHOW FULL PROCESSLIST;
SELECT * FROM information_schema.processlist WHERE TIME > 10;

-- PostgreSQL
SELECT pid, query, now() - query_start AS duration 
FROM pg_stat_activity 
WHERE state = 'active' AND now() - query_start > INTERVAL '10 seconds';

-- 2. Проанализировать план
EXPLAIN (ANALYZE, BUFFERS, VERBOSE) 
SELECT * FROM table WHERE condition;

-- 3. Проверить использование индексов
-- Удалить неиспользуемые индексы
```

#### **Оптимизация запросов:**
*   Использовать EXPLAIN перед оптимизацией
*   Фильтровать на ранних этапах (до JOIN, GROUP BY)
*   Использовать LIMIT для уменьшения объема
*   Избегать SELECT *, выбирать только нужные столбцы
*   Использовать подходящие типы JOIN
*   Разбивать сложные запросы на части

#### **Мониторинг и обслуживание:**

```sql
-- Обновление статистики
ANALYZE table_name;  -- PostgreSQL
ANALYZE TABLE table_name;  -- MySQL

-- Перестроение индексов
REINDEX INDEX index_name;  -- PostgreSQL
ALTER TABLE table_name ENGINE=InnoDB;  -- MySQL (неявное перестроение)

-- Мониторинг фрагментации
-- PostgreSQL
SELECT nspname, relname, round(100 * pg_relation_size(indexrelid) / pg_relation_size(indrelid)) / 100 AS index_ratio
FROM pg_index I LEFT JOIN pg_class C ON (C.oid = I.indexrelid) 
LEFT JOIN pg_namespace N ON (N.oid = C.relnamespace) 
WHERE nspname NOT IN ('pg_catalog', 'information_schema', 'pg_toast') AND C.relkind='i' 
AND pg_relation_size(indrelid) > 0;

-- Удаление неиспользуемых индексов (осторожно!)
SELECT schemaname, tablename, indexname 
FROM pg_stat_user_indexes 
WHERE idx_scan = 0;
```

#### **Ключевые метрики:**
*   **Время выполнения:** Должно быть стабильным и предсказуемым
*   **Количество обращений к диску:** Индексы должны уменьшать I/O
*   **Использование памяти:** Временные таблицы и сортировки в памяти быстрее
*   **Блокировки:** Долгосрочные блокировки тормозят систему


#### **Правила оптимизации:**
*   **Измеряй, потом оптимизируй:** Всегда используй EXPLAIN/EXPLAIN ANALYZE
*   **Индексы — палка о двух концах:** Ускоряют чтение, замедляют запись
*   **Составные индексы слева направо:** Порядок столбцов критически важен
*   **Статистика должна быть актуальной:** Иначе оптимизатор выберет плохой план
*   **Простота лучше сложности:** Часто простая оптимизация дает больший эффект
*   **Тестируй на реалистичных данных:** На 100 строках всё работает быстро